In [ ]:
# compute stats about number of tokens: total, trainable and not.
#

In [24]:
from datasets import load_dataset
import json
from  tqdm import tqdm
import pandas as pd

In [12]:
dataset = load_dataset("xaynetwork/agent_train_v1")

In [20]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("zai-org/GLM-4.5-Air", use_fast=True)

In [17]:
dataset["train"]["messages"][0][2]

{'content': '\n\nIch werde zur Beantwortung dieser Frage zunächst die einschlägige Gesetzesvorschrift recherchieren und dann nach Kommentaren und Gerichtsentscheidungen suchen, um die Anforderungen an die Tätigkeit des Täters zu konkretisieren.\n',
 'name': None,
 'reasoning_content': 'Die Frage betrifft die Anforderungen an die Tätigkeit des Täters bei der Überwindung einer besonderen Sicherung gegen Wegnahme im Sinne des § 243 I 2 Nr. 2 StGB. Es handelt sich um eine spezifische Frage zum deutschen Strafrecht, insbesondere zum besonders schweren Fall des Diebstahls.\n\nIch sollte zunächst die genaue Vorschrift nachschlagen, um den Wortlaut zu haben, und dann nach Kommentaren und Gerichtsentscheidungen suchen, die die Anforderungen an die Überwindung einer besonderen Sicherung konkretisieren.\n\nZuerst schaue ich mir den § 243 StGB an.',
 'role': 'assistant',
 'tool_call_id': None,
 'tool_calls': [{'function': {'arguments': '{"list_of_laws": ["stgb 243"]}',
    'name': 'german_law_look

In [31]:
new_data = []
for sample in tqdm(dataset["train"]):
    tmp = {"total": 0, "trainable": 0, "non-trainable": 0}
    for mes in sample["messages"]:
        if mes["role"] == "system" or mes["role"] == "user" or mes["role"] == "tool":
          tmp["non-trainable"] += len(tokenizer(mes["content"])["input_ids"])
        elif mes["role"] == "assistant":
            if mes["content"]:
              tmp["trainable"] += len(tokenizer(mes["content"])["input_ids"])
            if mes["reasoning_content"]:
                tmp["trainable"] += len(tokenizer(mes["reasoning_content"])["input_ids"])
            if mes["tool_calls"]:
                tmp["trainable"] += len(tokenizer(json.dumps(mes["tool_calls"]))["input_ids"])
        else:
          continue
    tmp["total"] = tmp["trainable"] + tmp["non-trainable"]
    new_data.append(tmp)


100%|██████████| 12995/12995 [08:14<00:00, 26.29it/s]


In [32]:
df = pd.DataFrame(new_data)
df.head()

,total,trainable,non-trainable
0,28266,2874,25392
1,46239,3376,42863
2,38165,3027,35138
3,21590,2560,19030
4,24073,2591,21482


In [33]:
df.to_csv("data_info.csv", index=False)

In [36]:
df["total"].describe()

count     12995.000000
mean      29589.859561
std       10597.725128
min         557.000000
25%       23344.000000
50%       28109.000000
75%       34792.500000
max      110321.000000
Name: total, dtype: float64

In [39]:
df["total"].quantile(0.99)

61700.97999999996

In [21]:
COLUMNS_TO_REMOVE = ["document_name"]  # add more keys here if needed

def remove_keys_from_messages(example):
    messages = example.get("messages", [])
    for m in messages:
        for key in COLUMNS_TO_REMOVE:
            if key in m:
                m.pop(key)
    example["messages"] = messages
    return example

# Apply
mdataset = dataset.map(remove_keys_from_messages, num_proc=8)

Map (num_proc=8):   0%|          | 0/12995 [00:00<?, ? examples/s]

In [3]:
def convert_to_glaive_format(input_data, output_file="train_data.json"):
    """
    Convert conversational data with reasoning and tool calls to Glaive format.

    Expected input format:
    - System message (optional)
    - User prompt (role='user')
    - Reasoning (optional, in reasoning_content)
    - Assistant response (role='assistant')
    - Tool calls (optional, in tool_calls)
    - Tool results (optional, role='tool')

    Output Glaive format:
    - from: "system" | "human" | "gpt" | "function_call" | "observation"
    - value: content string
    - tools: JSON string of available tools (at conversation level)
    """

    glaive_data = []
    for sample in input_data:
        current_conversation = []
        for msg in sample:
            role = msg.get('role')
            content = msg.get('content', '')
            reasoning = msg.get('reasoning_content')
            tool_calls = msg.get('tool_calls')

            if role == 'system':
                # System message
                # continue
                current_conversation.append({
                    "from": "system",
                    "value": content
                })

            elif role == 'user':
                current_conversation.append({
                    "from": "user",
                    "value": content
                })

            elif role == 'assistant':

                # Build assistant message with reasoning, content, and tool calls merged
                assistant_parts = []

                # 1. Add thinking/reasoning
                if reasoning:
                    assistant_parts.append(f"<|thinking|>\n{reasoning}\n<|/thinking|>")

                # 2. Add main content
                if content.strip():
                    assistant_parts.append(content)

                # Handle tool calls
                if tool_calls:

                    for tool_call in tool_calls:
                        function_data = tool_call.get('function', {})
                        function_name = function_data.get('name')
                        function_args = function_data.get('arguments', '{}')

                        # Parse arguments if string
                        if isinstance(function_args, str):
                            try:
                                function_args_dict = json.loads(function_args)
                            except:
                                function_args_dict = {}
                        else:
                            function_args_dict = function_args

                        # Format tool call in GLM-4.5 style
                        tool_call_text = f"<|tool_call|>\n{{\n  \"name\": \"{function_name}\",\n  \"arguments\": {json.dumps(function_args_dict, ensure_ascii=False)}\n}}\n<|/tool_call|>"
                        assistant_parts.append(tool_call_text)

                # Combine all parts
                assistant_value = "\n\n".join(assistant_parts)

                if assistant_value.strip():
                    current_conversation.append({
                        "from": "assistant",
                        "value": assistant_value
                    })

            elif role == 'tool':
                # Tool response
                current_conversation.append({
                    "from": "tool",
                    "value": content
                })

        # Create final conversation object
        if current_conversation:
            conversation_obj = {
                "conversations": current_conversation,
                "tools": json.loads("[{\"name\": \"calculate_area\", \"description\": \"Calculate the area of a given shape\", \"parameters\": {\"type\": \"object\", \"properties\": {\"shape\": {\"type\": \"string\", \"description\": \"The shape for which to calculate the area\"}, \"dimensions\": {\"type\": \"array\", \"items\": {\"type\": \"number\"}, \"description\": \"The dimensions required to calculate the area\"}}, \"required\": [\"shape\", \"dimensions\"]}}, {\"name\": \"calculate_discount\", \"description\": \"Calculate the discounted price based on original price and discount percentage\", \"parameters\": {\"type\": \"object\", \"properties\": {\"original_price\": {\"type\": \"number\", \"description\": \"The original price of the item\"}, \"discount_percentage\": {\"type\": \"number\", \"description\": \"The discount percentage\"}}, \"required\": [\"original_price\", \"discount_percentage\"]}}]")
            }

            glaive_data.append(conversation_obj)

    # Save to JSON file
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(glaive_data, f, ensure_ascii=False, indent=2)

    print(f"Converted {len(glaive_data)} conversations to {output_file}")
    # return glaive_data


In [4]:
convert_to_glaive_format(dataset["train"][0:4]["messages"], output_file="data/traces-test.json")

Converted 4 conversations to data/traces-test.json


In [5]:
convert_to_glaive_format(dataset["train"][:250]["messages"], output_file="data/traces.json")

Converted 250 conversations to data/traces.json


In [3]:
import json
from typing import List, Dict, Any, Optional
from datasets import load_dataset

# Define your tools schema - update this with your actual tools
TOOLS_SCHEMA = [
    {
        "type": "function",
        "function": {
            "name": "german_law_search",
            "description": "Search German law documents and regulations",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query for German law"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "german_commentary_search",
            "description": "Search German legal commentary and analysis",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query for German legal commentary"
                    }
                },
                "required": ["query"]
            }
        }
    }
]


def convert_to_axolotl_tool_format(
        conversations: List[Dict[str, Any]],
        tools: List[Dict[str, Any]] = None
) -> Dict[str, Any]:
    """
    Convert your dataset format to Axolotl's native tool calling format.

    Args:
        conversations: List of conversation turns with your current format
        tools: List of tool schemas (if None, uses TOOLS_SCHEMA)

    Returns:
        Dictionary in Axolotl's tool calling format
    """
    if tools is None:
        tools = TOOLS_SCHEMA

    messages = []

    for turn in conversations:
        role = turn["role"]

        if role == "user":
            # User message - simple format
            messages.append({
                "role": "user",
                "content": turn["content"]
            })

        elif role == "assistant":
            # Assistant message - may have reasoning and/or tool calls

            if turn.get("tool_calls"):
                # Assistant is making tool calls
                tool_calls = []
                for tc in turn["tool_calls"]:
                    # Parse the arguments string to JSON
                    args_str = tc["function"]["arguments"]
                    try:
                        args_dict = json.loads(args_str)
                    except json.JSONDecodeError:
                        # If already a dict or parsing fails, use as is
                        args_dict = args_str if isinstance(args_str, dict) else {"raw": args_str}

                    tool_calls.append({
                        "id": tc["id"],
                        "type": "function",
                        "function": {
                            "name": tc["function"]["name"],
                            "arguments": args_dict
                        }
                    })

                # Build the message
                message = {
                    "role": "assistant",
                    "tool_calls": tool_calls
                }

                # Add content if present (reasoning or text before tool call)
                if turn.get("content") or turn.get("reasoning_content"):
                    content_parts = []

                    if turn.get("reasoning_content"):
                        content_parts.append(turn["reasoning_content"])

                    if turn.get("content") and turn["content"].strip():
                        content_parts.append(turn["content"])

                    if content_parts:
                        message["content"] = "\n\n".join(content_parts)

                messages.append(message)

            else:
                # Assistant message without tool calls (final response)
                content_parts = []

                if turn.get("reasoning_content"):
                    content_parts.append(turn["reasoning_content"])

                if turn.get("content"):
                    content_parts.append(turn["content"])

                messages.append({
                    "role": "assistant",
                    "content": "\n\n".join(content_parts) if content_parts else ""
                })

        elif role == "tool":
            # Tool response message
            messages.append({
                "role": "tool",
                "tool_call_id": turn.get("tool_call_id", ""),
                "name": turn.get("name", ""),
                "content": turn["content"]
            })

    return {
        "tools": tools,
        "messages": messages
    }


def process_hf_dataset(
        dataset_name: str,
        output_file: str,
        split: str = "train",
        conversation_column: str = "messages",
        tools: List[Dict[str, Any]] = None,
        subset: Optional[str] = None,
        token: Optional[str] = None,
        max_samples: Optional[int] = None
):
    """
    Load dataset from Hugging Face and convert to Axolotl tool format.

    Args:
        dataset_name: HuggingFace dataset name (e.g., "username/dataset-name")
        output_file: Path to output JSONL file in Axolotl format
        split: Dataset split to use (default: "train")
        conversation_column: Name of the column containing conversation data
        tools: List of tool schemas (if None, uses TOOLS_SCHEMA)
        subset: Optional dataset subset/configuration name
        token: Optional HuggingFace token for private datasets
        max_samples: Maximum number of samples to process (None = all)
    """
    print(f"Loading dataset: {dataset_name}")

    # Load the dataset from HuggingFace
    if subset:
        dataset = load_dataset(dataset_name, subset, split=split, token=token)
    else:
        dataset = load_dataset(dataset_name, split=split)

    total_samples = len(dataset)
    # Limit dataset if max_samples is specified
    if max_samples is not None and max_samples < total_samples:
        dataset = dataset.select(range(max_samples))
        print(f"Dataset loaded: {total_samples} examples (processing first {max_samples})")
    else:
        print(f"Dataset loaded: {total_samples} examples")


    # Process and write to output file
    converted_count = 0
    skipped_count = 0

    with open(output_file, 'w', encoding='utf-8') as f_out:
        for idx, example in enumerate(dataset):
            try:
                # Extract conversation data
                if conversation_column in example:
                    conversation_turns = example[conversation_column]
                elif isinstance(example, list):
                    conversation_turns = example
                else:
                    # Try to find a column that looks like messages
                    possible_columns = ['messages', 'conversations', 'turns', 'dialogue']
                    conversation_turns = None
                    for col in possible_columns:
                        if col in example:
                            conversation_turns = example[col]
                            break

                    if conversation_turns is None:
                        print(f"Warning: Could not find conversation column in example {idx}, skipping")
                        skipped_count += 1
                        continue

                # Handle different structures
                if not isinstance(conversation_turns, list):
                    print(f"Warning: Conversation data at index {idx} is not a list, skipping")
                    skipped_count += 1
                    continue

                # Convert to Axolotl format
                converted = convert_to_axolotl_tool_format(conversation_turns, tools)

                # Write to output
                f_out.write(json.dumps(converted, ensure_ascii=False) + '\n')
                converted_count += 1

                if (idx + 1) % 100 == 0:
                    print(f"Processed {idx + 1}/{len(dataset)} examples...")

            except Exception as e:
                print(f"Error processing example {idx}: {e}")
                skipped_count += 1
                continue

    print(f"\nConversion complete!")
    print(f"Successfully converted: {converted_count}")
    print(f"Skipped: {skipped_count}")
    print(f"Output saved to: {output_file}")


In [18]:
# Process the HuggingFace dataset
process_hf_dataset(
    dataset_name="xaynetwork/agent_train_v1",
    output_file="data/traces-axolotl-t.jsonl",
    split="train",
    max_samples=5,
)

Loading dataset: xaynetwork/agent_train_v1
Dataset loaded: 12995 examples (processing first 250)
Processed 100/250 examples...
Processed 200/250 examples...

Conversion complete!
Successfully converted: 250
Skipped: 0
Output saved to: data/traces-axolotl.jsonl


In [1]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("zai-org/GLM-4.5-Air")


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/20.0M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [ ]:
from datasets import load_from_disk

# Load tokenizer compatible with your base model
tokenizer = AutoTokenizer.from_pretrained("zai-org/GLM-4.5-Air")

# Load the preprocessed dataset
# (Assumes default `last_run_prepared/` folder)
ds = load_from_disk("/home/mryakhovskiy/axolotl/models/docker/data_local/2016f592b8a69c7242db9955d0d97c35")  # adjust path if needed

# Inspect one example
row = ds[0]
input_ids = row["input_ids"]
labels = row["labels"]

# Decode tokens to text
decoded = tokenizer.decode(input_ids)
print("Decoded prompt:", decoded)

# # Show tokens + labels in a readable form
# for token_id, label in zip(input_ids, labels):
#     token = tokenizer.decode([token_id])
#     print(f"Token: {token!r}, id: {token_id}, label: {label}")


In [42]:
def function_str_json(entry):
    for m in entry["messages"]:
        try:
            if m["tool_calls"]:
                for func in m["tool_calls"]:
                    func["function"]["arguments"] = json.loads(func["function"]["arguments"])
        except:
            print(m)
            continue
    return entry

def is_valid(entry):
    try:
        # Try to run your conversion
        function_str_json(entry)
        return True
    except Exception:
        return False


def convert_arguments_to_json(example):
    for message in example['messages']:
        if message.get('tool_calls'):
            for tool_call in message['tool_calls']:
                try:
                    args_str = tool_call['function']['arguments']
                    if isinstance(args_str, str):
                        tool_call['function']['arguments'] = json.loads(args_str)
                except (json.JSONDecodeError, KeyError, TypeError):
                    # Skip this tool_call if it fails
                    print()
                    continue
    return example

In [40]:
subset = dataset["train"].select(range(1000))

In [43]:
mdataset = subset.map(function_str_json, num_proc=1, load_from_cache_file=False)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [44]:
type(mdataset)
mdataset.to_json("./data/traces-ax.jsonl")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

110487927

In [108]:
mdataset[0]

{'messages': [{'content': "You are Noxtua, a legal AI assistant.\nYou assist legal professionals with their tasks such as research, drafting, and legal analysis.\nYou may use tools when appropriate to achieve your goals.\nYou are advising for the jurisdiction of Germany. You have access to data from Beck.\nNote: User messages may contain <environment> tags that are automatically added by the system to inform you about state changes (e.g., new files uploaded, canvas status). These are not part of the user's actual message.\nCore behavior:\n- Be precise, concise, and technically accurate.\n- Your style should be academic. Avoid using bullet points.\n- Use markdown formatting. Organize your answer with sections, headings, etc.\n- Use tools to gather authoritative content for statutes, cases, and commentary when needed.\n- Commentaries and court decisions are important to understand how the law is interpreted by the courts and practitioners. When in doubt, check the commentaries and court 

In [3]:
res = tokenizer.apply_chat_template(dd["messages"], tokenize=False)

In [8]:
tokenizer.eos_token

'<|endoftext|>'

In [4]:
res

'[gMASK]<sop><|system|>\nYou are Noxtua, a legal AI assistant.\nYou assist legal professionals with their tasks such as research, drafting, and legal analysis.\nYou may use tools when appropriate to achieve your goals.\nYou are advising for the jurisdiction of Germany. You have access to data from Beck.\nNote: User messages may contain <environment> tags that are automatically added by the system to inform you about state changes (e.g., new files uploaded, canvas status). These are not part of the user\'s actual message.\nCore behavior:\n- Be precise, concise, and technically accurate.\n- Your style should be academic. Avoid using bullet points.\n- Use markdown formatting. Organize your answer with sections, headings, etc.\n- Use tools to gather authoritative content for statutes, cases, and commentary when needed.\n- Commentaries and court decisions are important to understand how the law is interpreted by the courts and practitioners. When in doubt, check the commentaries and court d

In [2]:
dd = {'messages': [{'content': "You are Noxtua, a legal AI assistant.\nYou assist legal professionals with their tasks such as research, drafting, and legal analysis.\nYou may use tools when appropriate to achieve your goals.\nYou are advising for the jurisdiction of Germany. You have access to data from Beck.\nNote: User messages may contain <environment> tags that are automatically added by the system to inform you about state changes (e.g., new files uploaded, canvas status). These are not part of the user's actual message.\nCore behavior:\n- Be precise, concise, and technically accurate.\n- Your style should be academic. Avoid using bullet points.\n- Use markdown formatting. Organize your answer with sections, headings, etc.\n- Use tools to gather authoritative content for statutes, cases, and commentary when needed.\n- Commentaries and court decisions are important to understand how the law is interpreted by the courts and practitioners. When in doubt, check the commentaries and court decisions.\n- If the user referenced documents, or requests information which might require document's content, use document tools to list and read them before answering. There might be multiple documents referenced or uploaded across session of single message.\n- Prefer exact lookup tools when the user cites a specific provision.\n- If sources are insufficient, say so clearly and avoid speculation.\n- Make sure your answers are verifiable claims. Meaning they are based on the sources provided by the tools.\n- If you make your own conclusions, deductions or inferences, make that clear in your answer.\n- You should always cite your sources by generating a citation like <citation>id_for_citations</citation>. Taking the id_for_citations from the search tool call result.\n- For example, a valid citation would be <citation>abcd1234</citation>.\n- If you want to cite multiple sources, you can use multiple citations like <citation>abcd1234</citation><citation>efgh5678</citation>.\n- DO NOT cite using law titles or book numbers or commentary titles, only use the document ids.\n- You may put the citations anywhere in your answer including within sentences and paragraphs.\nUnless otherwise instructed, answer in GERMAN.",
               'name': None,
               'reasoning_content': None,
               'role': 'system',
               'tool_call_id': None,
               'tool_calls': None},
              {'content': 'Welche Anforderungen werden an die Tätigkeit des Täters bei der Überwindung einer besonderen Sicherung gegen Wegnahme im Sinne des § 243 I 2 Nr. 2 StGB gestellt?',
               'name': None,
               'reasoning_content': None,
               'role': 'user',
               'tool_call_id': None,
               'tool_calls': None},
              {'content': '\n\nIch werde zur Beantwortung dieser Frage zunächst die einschlägige Gesetzesvorschrift recherchieren und dann nach Kommentaren und Gerichtsentscheidungen suchen, um die Anforderungen an die Tätigkeit des Täters zu konkretisieren.\n',
               'name': None,
               'reasoning_content': 'Die Frage betrifft die Anforderungen an die Tätigkeit des Täters bei der Überwindung einer besonderen Sicherung gegen Wegnahme im Sinne des § 243 I 2 Nr. 2 StGB. Es handelt sich um eine spezifische Frage zum deutschen Strafrecht, insbesondere zum besonders schweren Fall des Diebstahls.\n\nIch sollte zunächst die genaue Vorschrift nachschlagen, um den Wortlaut zu haben, und dann nach Kommentaren und Gerichtsentscheidungen suchen, die die Anforderungen an die Überwindung einer besonderen Sicherung konkretisieren.\n\nZuerst schaue ich mir den § 243 StGB an.',
               'role': 'assistant',
               'tool_call_id': None,
               'tool_calls': [{'function': {'arguments': {'court_instances': None,
                                                          'laws': None,
                                                          'list_of_laws': ['stgb 243'],
                                                          'query': None},
                                            'name': 'german_law_lookup'},
                               'id': 'call_b42d5b2e87b0475eaa5bc1c4',
                               'type': 'function'}]},
              {'content': '<GermanLaw>\n  <id_for_citations>4BwovW5b</id_for_citations>\n  <title>§ 243 StGB</title>\n  <text>§ 243 Besonders schwerer Fall des Diebstahls\n(1) 1 In besonders schweren Fällen wird der Diebstahl mit Freiheitsstrafe von drei Monaten bis zu zehn Jahren bestraft. 2 Ein besonders schwerer Fall liegt in der Regel vor, wenn der Täter 1. zur Ausführung der Tat in ein Gebäude, einen Dienst- oder Geschäftsraum oder in einen anderen umschlossenen Raum einbricht, einsteigt, mit einem falschen Schlüssel oder einem anderen nicht zur ordnungsmäßigen Öffnung bestimmten Werkzeug eindringt oder sich in dem Raum verborgen hält,\n2. eine Sache stiehlt, die durch ein verschlossenes Behältnis oder eine andere Schutzvorrichtung gegen Wegnahme besonders gesichert ist,\n3. gewerbsmäßig stiehlt,\n4. aus einer Kirche oder einem anderen der Religionsausübung dienenden Gebäude oder Raum eine Sache stiehlt, die dem Gottesdienst gewidmet ist oder der religiösen Verehrung dient,\n5. eine Sache von Bedeutung für Wissenschaft, Kunst oder Geschichte oder für die technische Entwicklung stiehlt, die sich in einer allgemein zugänglichen Sammlung befindet oder öffentlich ausgestellt ist,\n6. stiehlt, indem er die Hilflosigkeit einer anderen Person, einen Unglücksfall oder eine gemeine Gefahr ausnutzt oder\n7. eine Handfeuerwaffe, zu deren Erwerb es nach dem Waffengesetz der Erlaubnis bedarf, ein Maschinengewehr, eine Maschinenpistole, ein voll- oder halbautomatisches Gewehr oder eine Sprengstoff enthaltende Kriegswaffe im Sinne des Kriegswaffenkontrollgesetzes oder Sprengstoff stiehlt.\n(2) In den Fällen des Absatzes\u20041 Satz\u20042 Nr.\u20041 bis\u20046 ist ein besonders schwerer Fall ausgeschlossen, wenn sich die Tat auf eine geringwertige Sache bezieht.</text>\n</GermanLaw>',
               'name': 'german_law_lookup',
               'reasoning_content': None,
               'role': 'tool',
               'tool_call_id': 'call_b42d5b2e87b0475eaa5bc1c4',
               'tool_calls': None},
              {'content': '\n\n',
               'name': None,
               'reasoning_content': 'Jetzt habe ich den Wortlaut des § 243 I 2 Nr. 2 StGB. Es geht um die besonders schweren Fälle des Diebstahls, wenn der Täter "eine Sache stiehlt, die durch ein verschlossenes Behältnis oder eine andere Schutzvorrichtung gegen Wegnahme besonders gesichert ist".\n\nDie Frage zielt speziell auf die Anforderungen ab, die an die Tätigkeit des Täters bei der "Überwindung" einer solchen Sicherung gestellt werden. Das Wort "Überwindung" kommt im Gesetzestext nicht explizit vor, aber es wird in der Rechtsprechung und Literatur verwendet, um beschreiben zu müssen, was der Täter tun muss, um an eine besonders gesicherte Sache zu gelangen.\n\nIch sollte jetzt nach Kommentaren und Gerichtsentscheidungen suchen, um herauszufinden, welche konkreten Anforderungen an die Überwindungstätigkeit gestellt werden.',
               'role': 'assistant',
               'tool_call_id': None,
               'tool_calls': [{'function': {'arguments': {'court_instances': None,
                                                          'laws': ['stgb 243'],
                                                          'list_of_laws': None,
                                                          'query': 'Überwindung besonderer Sicherung § 243 Abs 1 Nr 2 StGB Anforderungen Täter Tätigkeit'},
                                            'name': 'german_commentary_search'},
                               'id': 'call_68edba1c42a846fba6affee0',
                               'type': 'function'}]},
              {'content': '<GermanCommentary>\n  <id_for_citations>NgZaiqGX</id_for_citations>\n  <title>Wittig - BeckOK StGB | StGB § 243 Rn. 18-19</title>\n  <text>Rechtsnormen: § 243 StGB\nRechtsgebiete: StrafR, allgemein, HaWiR, ArbSozR, ZivilR, SteuR, OeR\n4. Besondere Sicherung gegen Wegnahme\n18 Das verschlossene Behältnis oder die andere Schutzvorrichtung muss eine besondere Sicherung gegen Wegnahme darstellen.\n18.1 Eine besondere Sicherung fehlt, wenn die Vorrichtung die Wegnahme nicht wesentlich erschwert (OLG Hamm NStZ-RR 2009, 204), zB, wenn der Schlüssel steckt, neben dem Behältnis liegt oder leicht erreichbar ist (Bachmann/Goeck StV 2011, 19; aA in einem obiter dictum KG NJW 2012, 1093 (1094)), wenn der Täter den Schlüssel rechtmäßig besitzt (MüKoStGB/Schmitz Rn. 36 mwN) oder wenn das Behältnis (erkennbar) unproblematisch geöffnet werden kann (OLG Hamm NStZ 2009, 204; MüKoStGB/Schmitz Rn. 36). Auf eine Kraftentfaltung kommt es nicht an, ob und wie der Täter die Schutzvorrichtung beseitigt, ist irrelevant (OLG Hamm NStZ 2009, 204; OLG Frankfurt a.M. NJW 1998, 3028).\n18.2 Eine besondere Sicherung fehlt auch, wenn eine Vorkehrung anderen Zwecken dient, zB beim Verschnüren dem Zusammenhalt (MüKoStGB/Schmitz Rn. 35 mwN), dem Schutz vor Verlust (Tasche mit Reißverschluss, zugeknöpfte Hosentasche, Sicherungen bei Schmuck, Festschnallen auf dem Fahrradgepäckträger) oder vor Kenntnisnahme (Briefumschlag oder Briefsiegel MüKoStGB/Schmitz Rn. 36; OLG Stuttgart NJW 1964, 738; aA RGSt 54, 295; zum Ganzen vgl. TK-StGB/Bosch Rn. 24; Fischer/Fischer Rn. 16; WHS StrafR BT II Rn. 245 jew. mwN). Das besondere Gewicht oder die Unhandlichkeit der Sache reicht idR nicht aus (MüKoStGB/Schmitz Rn. 35). Auch die Halterung eines Autoradios ist keine besondere Sicherung gegen Wegnahme (OLG Schleswig NJW 1984, 67 (68)). Es reicht aus, dass dem Sicherungszweck neben anderen Zwecken (zB Schutz vor Beschädigungen, Verpackung) zumindest eine nicht unerhebliche Rolle zukommt, so zB bei einem mit Klebestreifen verschlossenen Karton (OLG Hamm NJW 1978, 769). Eingebaute Signale oder sonstige Vorkehrungen reichen dagegen nicht aus, wenn sie lediglich der nachträglichen Entdeckung des Täters bzw. der Wiedererlangung der gestohlenen Sache dienen sollen (OLG Stuttgart NStZ 1985, 76; OLG Dresden NStZ-RR 2015, 211 mAnm Hecker JuS 2015, 847; TK-StGB/Bosch Rn. 24 mwN). Dies gilt auch für elektronische Sicherungsetiketten zB an Waren (BGH NStZ 2019, 212 mBespr Jahn JuS 2018, 103; OLG Dresden NStZ-RR 2015, 211 mAnm Hecker JuS 2015, 847; BayObLG NJW 1995, 3000 (3001); OLG Düsseldorf NJW 1998, 1002; Zopfs JURA 2007, 421 (425); WHS StrafR BT II Rn. 246 mwN; aA TK-StGB/Bosch Rn. 24 mwN), wobei hier ein unbenannter bes. schwerer Fall in Betracht kommt (OLG Dresden NStZ-RR 2015, 211 (212) mAnm Hecker JuS 2015, 847). Der BGH hat dies auch für sog. Sicherungsspinnen (die Verpackung einfassende Elektrodrähte, die bei Durchtrennung oder beim Passieren des Kassenbereichs ein Alarmsignal auslösen) angenommen, wenn sie erst bei Verlassen des Geschäfts Alarm auslösen. Lösen sie schon bei der Durchtrennung einen Alarm aus, ist zu prüfen, ob die Wegnahme dadurch wesentlich erschwert worden ist (BGH NStZ 2019, 212 (212)).\n18.3 Bei Spielautomaten (→ § 242 Rn. 22.1) liegt Nr. 2 nur vor, wenn die Sicherungsvorrichtung überwunden wird (zB durch Aufbrechen oder regelwidriger Manipulation von außen, BayObLG NJW 1981, 2826), nicht aber ohne äußere Manipulation durch Überlistung (zB durch Einwurf von Falschgeld) umgangen wird (OLG Düsseldorf NStZ 1999, 248 (249)); umfassend dazu WHS StrafR BT II Rn. 247 mwN. Nr. 2 greift auch nicht bei der Überlistung eines Geldwechselautomaten durch präparierte Geldscheine (OLG Düsseldorf NJW 2000, 158 (159); Fischer/Fischer Rn. 16 mwN; krit. Biletzki NStZ 2000, 424).\n18.4 Noch nicht endg. geklärt ist, ob auch die missbräuchliche Verwendung von richtigen Schlüsseln, Codes und Geheimnummern unter Nr. 2 fällt (verneinend OLG Zweibrücken NStZ-RR 2018, 249 (250) mwN; s. auch Rengier StrafR BT I § 3 Rn. 27 mwN). Die hM bejaht das Regelbeispiel jedenfalls dann, wenn der Schlüssel etc unbefugt insbes. durch eine Straftat erlangt wurde (BGH NStZ 2011, 36; BayObLG NJW 1987, 665 (666); s. auch OLG Hamm NJW 1982, 777; Eisele StrafR BT II Rn. 116). Zur unbefugten Öffnung einer Registrierkasse mittels eines Nothebels oder einer geheimen Schnellöffnungstaste s. OLG Frankfurt a.M. NJW 1988, 3028; AG Freiburg NJW 1994, 400 mablAnm Murmann NJW 1995, 935; hierzu auch Rengier StrafR BT I § 3 Rn. 28 mwN.\n19 Nr. 2 liegt nach der hM auch vor, wenn die Sache samt Sicherungsvorrichtung entwendet wird, um sie erst später zu öffnen (BGHSt 24, 248; SSW StGB/Kudlich Rn. 21; Fischer/Fischer Rn. 17 mwN; diff. MüKoStGB/Schmitz Rn. 37). Gerade bei kleinen oder sonstigen Behältnissen, die problemlos insgesamt weggenommen werden können, ist jedoch fraglich, ob diese überhaupt die Sache vor Wegnahme schützen können (NK-StGB/Kindhäuser/Hoven Rn. 24).</text>\n</GermanCommentary>\n<GermanCommentary>\n  <id_for_citations>MvxOpSVo</id_for_citations>\n  <title>Wittig - BeckOK StGB | StGB § 243 Rn. 30-31.1</title>\n  <text>Rechtsnormen: § 243 StGB\nRechtsgebiete: StrafR, allgemein, HaWiR, ArbSozR, ZivilR, SteuR, OeR\nF. Versuch\nI. Strafbarkeit des Versuchs\n30 Da § 243 nach ganz hM (→ Rn. 1) kein Tatbestand ist, gibt es keinen Versuch des § 243. Der Beginn der Verwirklichung des Regelbeispiels begr. für sich allein noch keinen Versuch des § 242 (Eisele StrafR BT II Rn. 144). Dagegen kann bereits der Versuch des § 242 ein bes. schwerer Fall sein (BGHSt 33, 370; LK-StGB/Vogel/Brodowski Rn. 72). Dies ist unproblematisch, wenn bei einem Diebstahlsversuch die Regelbeispielsmerkmale (insbes. nach Nr. 1 u. 2) bereits vollständig verwirklicht sind (BGHSt 33, 370; LK-StGB/Vogel/Brodowski Rn. 72 mwN; Huber JuS 2016, 597 (598)). Nach BGHSt 33, 370 soll ein versuchter Diebstahl in einem bes. schweren Fall aber auch vorliegen, wenn der Diebstahl und die beabsichtigte Verwirklichung des Regelbeispiels nur versucht werden (BayObLG NStZ 1997, 442; zu Recht krit. Zopfs JURA 2007, 421 (423); WHS StrafR BT II Rn. 223 ff.; LK-StGB/Vogel/Brodowski Rn. 73; aA MüKoStGB/Schmitz Rn. 89 jew. mwN; Huber JuS 2016, 597 (598); diff. nach Handlungs- u. Erfolgsunrecht Franzke NStZ 2018, 566). In den (seltenen) Fällen, in denen der Diebstahl vollendet und die beabsichtigte Verwirklichung des Regelbeispiels ausbleibt, wendet die wohl hM nur den Strafrahmen des § 242 an (offen gelassen von BGHSt 33, 370; BGH NStZ-RR 1997, 293 zu § 176 S. 2 Nr. 1 aF; BGH NStZ 2003, 602 zu § 177 Abs. 2 S. 2 Nr. 1; LK-StGB/Vogel/Brodowski Rn. 74 mwN; aA NK-StGB/Kindhäuser/Hoven Rn. 46 f. mwN; Huber JuS 2016, 597 (598)). Zu den Auswirkungen auf den Strafrahmen bei der Annahme eines Versuchs in einem bes. schweren Fall vgl. LK-StGB/Vogel/Brodowski Rn. 77.\nII. Versuchsbeginn\n31 Nach § 22 kommt es nach zutr. Ansicht für den Versuchsbeginn allein auf das unmittelbare Ansetzen zur Verwirklichung des Grundtatbestandes des § 242 ("Ansetzen zur Wegnahme") an (BGH NStZ 2017, 86 mAnm Engländer; Fischer/Fischer Rn. 28 mwN; zum Versuchsbeginn bei § 242 → § 242 Rn. 44). Nach der neueren Rspr. ist danach allerdings nur das Erfordernis krit. Prüfung umschrieben, ob der Täter im jeweiligen Einzelfall nach seiner Vorstellung von der Tat bereits die Schwelle überschritten hat, die ‒ ohne weitere Zwischenakte ‒ in die vollständige Verwirklichung des Tatbestands münden soll (BGH NStZ 2020, 353 (354) mAnm Kudlich u. mBespr Eisele 2020, 796). Deshalb ist es zumindest missverständlich, wenn formuliert wird, das strafbare Versuchsstadium sei stets erreicht, wenn der Täter zur Verwirklichung des Regelbeispiels ansetze (so aber OLG Hamm MDR 1976, 155; krit. WHS StrafR BT II Rn. 227 mwN). Dies kann der Fall sein, wenn dadurch der Täter nach seiner Vorstellung unmittelbar zur Verwirklichung des Diebstahls ansetzt, muss es aber nicht.\n31.1 So ist zB das Entriegeln eines Fensters, um in der folgenden Nacht einzusteigen, lediglich eine straflose Vorbereitungshandlung (LK-StGB/Vogel/Brodowski Rn. 75).</text>\n</GermanCommentary>\n<GermanCommentary>\n  <id_for_citations>z27iEiVC</id_for_citations>\n  <title>Wittig - BeckOK StGB | StGB § 243 Rn. 5-7</title>\n  <text>Rechtsnormen: § 243 StGB\nRechtsgebiete: StrafR, allgemein, HaWiR, ArbSozR, ZivilR, SteuR, OeR\n2. Umschlossener Raum\n5 Umschlossener Raum ist der Oberbegriff für die in Nr. 1 genannten sonstigen Räumlichkeiten. Darunter versteht man jedes Raumgebilde, das (mind. auch) dazu bestimmt ist, von Menschen betreten zu werden, und das mit (zumindest teilw. künstlichen) Vorrichtungen umgeben ist, die das Eindringen von Unbefugten abwehren sollen (BGHSt 1, 158 (164); TK-StGB/Bosch Rn. 8; Fischer/Fischer Rn. 4; Rengier StrafR BT I § 3 Rn. 10). Umschlossen bedeutet nicht notwendig verschlossen (RGSt 32, 141; BGHSt 1, 158 (164); BGH NJW 1954, 1897 (1898); TK-StGB/Bosch Rn. 8; Küper/Zopfs StrafR BT Rn. 424 mwN), sondern liegt bereits vor, wenn für den Täter tatsächlich ein Hindernis besteht. Dies setzt voraus, dass die Überwindung der Umgrenzung dem Täter nicht nur unerhebliche Mühe bereitet (BGH NStZ 1983, 168). Zudem muss der Wille erkennbar werden, durch die Vorrichtung Unbefugte am Eindringen zu hindern (BGH NJW 1954, 1897 (1898)), sodass Umgrenzungen, die anderen Zwecken dienen (zB Weidezäune), ausscheiden (BGH StV 1983, 149; SSW StGB/Kudlich Rn. 8). Der umschlossene Raum ist zu unterscheiden vom Behältnis iSd Nr. 2 (→ Rn. 15).\n5.1 Deshalb hebt das Vorhandensein eines unverschlossenen Eingangs oder einer solchen Zufahrt das Umschlossensein nicht auf, soweit nicht jedermann freien und ungehinderten Zugang hat (RGSt 32, 141; 54, 20; TK-StGB/Bosch Rn. 8 mwN) oder es nicht offensichtlich an einem Ausschlusswillen des Berechtigten fehlt, wie zB bei Telefonzellen (OLG Hamburg NJW 1962, 1453) oder öffentlichen Toiletten (WHS StrafR BT II Rn. 232). Wenn es an einem tatsächlichen Hindernis fehlt, zB beim Betreten durch vorhandene oder leicht herstellbare Lücken in einer Umfriedung (Hecke, Zaun) oder beim Übersteigen eines niedrigen Zauns, ist das Regelbeispiel ausgeschlossen (BGH StV 1984, 204; NStZ 2000, 143; Rengier StrafR BT I § 3 Rn. 11). Eine nur teilw. aus natürlichen Hindernissen bestehende Umgrenzung (Felsen, Gräben, Hecken), soweit sie tatsächlich ein Hindernis darstellt, genügt, nicht aber eine ausschl. natürliche Abgrenzung, zB bei einer Insel (MüKoStGB/Schmitz Rn. 14 mwN).\n6 Nicht maßgeblich sind Bodenverbundenheit – entgegen der früheren Rspr. des RG (RGSt 70, 360 (362)) – und Unbeweglichkeit (BGHSt 1, 158 (163)).\n6.1 Deshalb sind auch der Innenraum (nicht aber der Kofferraum, vgl. aber Nr. 2) eines Kfz (BGHSt 2, 214 (215), Schiffe (BGHSt 1, 158), Eisenbahnwagen, Wohnwagen (BGHSt 1, 158) und ein Lkw umschlossene Räume. Auch überdacht braucht der umschlossene Raum nicht zu sein; deshalb sind bei Bestehen entspr. Umschließungen auch Friedhöfe zur Nachtzeit (BGH NJW 1954, 1897), Höfe (RGSt 39, 104 (106)), Lagerplätze (BGH NStZ 2000, 143), Gärten und umzäunte Freigelände (BGH NStZ-RR 2005, 140 (141 f.)) umschlossene Räume.\n7 Als illustrierende Beispiele für umschlossene Räume, die damit auch deren Merkmale (insbes. Umschlossensein) erfüllen müssen, nennt das Gesetz " Gebäude " sowie " Diensträume " und " Geschäftsräume ". Ein Gebäude ist "ein durch Wände und Dach begrenztes, mit dem Erdboden fest – wenn auch nur durch die eigene Schwere – verbundenes Bauwerk, das den Eintritt von Menschen gestattet und das Unbefugte" abhalten soll (BGHSt 1, 158; NK-StGB/Kindhäuser/Hoven Rn. 11 mwN). Auch das Zirkuszelt ist ein Gebäude. Halbfertige Bauten oder Abbruchbauten sind Gebäude, wenn sie schon oder noch die zur Abwehr Unbefugter nötigen Umschließungen haben (Fischer/Fischer Rn. 4; anders bei §§ 305, 306). Auch Teile eines Gebäudes, wie zB Zimmer (BGHSt 1, 158 (167); weitere Bsp. bei TK-StGB/Bosch Rn. 9), sind umschlossene Räume. Dienst- oder Geschäftsräume sind Gebäudeteile, die dem Aufenthalt von Menschen während der Arbeitszeit bzw. zur Vornahme beruflicher oder sonstiger geschäftlicher Tätigkeit dienen (MüKoStGB/Schmitz Rn. 17 mwN). Für das Einbrechen etc in Wohnungen gilt § 244 Abs. 1 Nr. 3 (→ § 244 Rn. 21 ff.), in dauerhaft genutzte Privatwohnungen § 244 Abs. 4 (→ § 244 Rn. 25).</text>\n</GermanCommentary>\n<GermanCommentary>\n  <id_for_citations>s85NpErv</id_for_citations>\n  <title>Wittig - BeckOK StGB | StGB § 243 Rn. 10</title>\n  <text>Rechtsnormen: § 243 StGB\nRechtsgebiete: StrafR, allgemein, HaWiR, ArbSozR, ZivilR, SteuR, OeR\n4. Einsteigen\n10 Einsteigen ist jedes Hineingelangen in den umschlossenen Raum durch eine zum ordnungsgemäßen Eintritt nicht bestimmte Öffnung unter Überwindung von Hindernissen, die sich aus der Eigenart der Umschließung ergeben und die den Zugang erheblich erschweren (RGSt 55, 144; BGHSt 10, 132 (133); BGH NStZ-RR 2010, 374; BGHSt 61, 166; BGH BeckRS 2022, 23383; Küper/Zopfs StrafR BT Rn. 206 mwN). Der Täter muss die Umschließung mit Geschicklichkeit oder Kraft überwinden, ohne sie aufzubrechen oder zu beseitigen, zB durch Überklettern einer höheren Mauer oder das Eindringen durch ein Fenster (MüKoStGB/Schmitz Rn. 23 mwN). Das Benutzen eines verbotenen, aber offenen Eingangs genügt nicht (BGHSt 10, 132 (133); BGH NStZ-RR 2010, 374; Fischer/Fischer Rn. 6 mwN). Unerheblich ist auch, wenn der Täter die Räumlichkeit durch einen zum ordnungsgemäßen Eintritt bestimmten Eingang betritt, auf welche Weise er diesen geöffnet hat (BGHSt 61, 166 mAnm Theile ZJS 2016, 667). Eine "steigende" Bewegung ist nicht erforderlich (Küper/Zopfs StrafR BT Rn. 206), sodass zB auch das Hinab- oder Hineinkriechen (RGSt 13, 258) oder das Hindurchzwängen durch eine schmale Öffnung (RGSt 55, 144) ausreichen. Das bloße Hineingreifen und Herausholen von Sachen genügt (anders als beim Einbrechen) nicht; vielmehr muss sich der Täter im Raum einen festen körperlichen Stützpunkt geschaffen haben, von dem aus er die Wegnahme ausführt (BGH BeckRS 2014, 06840; NJW 1968, 1887; MüKoStGB/Schmitz Rn. 24; NK-StGB/Kindhäuser/Hoven Rn. 14; Küper/Zopfs StrafR BT Rn. 206 f. jew. mwN).</text>\n</GermanCommentary>\n<GermanCommentary>\n  <id_for_citations>jqFQ2Dnn</id_for_citations>\n  <title>Wittig - BeckOK StGB | StGB § 243 Rn. 1-3</title>\n  <text>Rechtsnormen: § 243 StGB\nRechtsgebiete: StrafR, allgemein, HaWiR, ArbSozR, ZivilR, SteuR, OeR\nA. Allgemeines\n1 § 243 regelt bes. schwere Fälle des Diebstahls (§ 242) nach der Regelbeispielmethode (dazu BGHSt 23, 254; 26, 104; 33, 370, zur Verfassungsmäßigkeit der Regelbeispielmethode BVerfGE 45, 363 (371 ff.); zu Recht krit. im Hinblick auf den Bestimmtheitsgrundsatz des Art. 103 Abs. 2 GG zB MüKoStGB/Schmitz Rn. 3 mwN). Die Erfüllung eines der in Abs. 1 S. 2 Nr. 1 – 7 aufgeführten Regelbeispiele macht einen Diebstahl nur "in der Regel" zu einem bes. schweren Fall, der zur Anwendung des – ggü. § 242 erhöhten – Strafrahmens von drei Monaten bis zu zehn Jahren führt. Die Regelbeispiele haben nur indizielle Bedeutung und enthalten keinen abschl. Katalog (BGHSt 23, 254 (257)). § 243 ist kein Qualifikationstatbestand des § 242 (hM BGHSt 23, 254 (256); 26, 104 (105); TK-StGB/Bosch Rn. 1 mwN; aA zB Eisele JA 2006, 309 (312)). Die Merkmale der Regelbeispiele sind Strafzumessungs-, nicht Tatbestandsmerkmale, auch wenn zu diesen, da sie einen ggü. dem Tatbestand des § 242 erhöhten Unrechts- und Schuldgehalt typisieren, kein "tief greifender Wesensunterschied" besteht, sie also tatbestandsähnlich sind (BGHSt 29, 359 (368); 33, 370 (374); TK-StGB/Bosch Rn. 1; diff. NK-StGB/Kindhäuser/Hoven Rn. 7; zu den Unterschieden SK-StGB/Hoyer Rn. 1, der von "Erschwernisgründen" sprechen will). § 243 hat keine "Strafbarkeitsbegründungs-, sondern nur Straftatfolgenrelevanz, genauer: Strafbemessungsrelevanz, noch genauer: Strafrahmenbestimmungsrelevanz" (Mitsch StrafR BT II S. 81 mwN; s. auch BGH NJW 2002, 150 (151)). Die Rechtsnatur des § 243 als Strafzumessungsregel hat strafprozessuale Konsequenzen, zB hins. der Verjährung (§ 78 Abs. 4, vgl. aber § 78b Abs. 4); vgl. ausf. Mitsch StrafR BT II S. 82. Zur Problematik des Versuchs → Rn. 30. Zur Gesetzesgeschichte s. MüKoStGB/Schmitz Rn. 2; NK-StGB/Kindhäuser/Hoven Rn. 1 f. jew. mwN; zur kriminalpolitischen Bedeutung s. MüKoStGB/Schmitz Rn. 8 f.\n2 Ist ein Regelbeispiel erfüllt, tritt die sog. Regelwirkung (Indizwirkung) ein. Eine richterliche Begründung für die Annahme eines bes. schweren Falls ist nicht erforderlich (Mitsch StrafR BT II S. 83). Die Regelwirkung kann widerlegt werden und zwar nach der hM unter Abwägung aller Zumessungstatsachen aufgrund einer Gesamtbewertung aller wesentlichen tat- und täterbezogenen Umstände (BGHSt 23, 254 (257); krit. SK-StGB/Hoyer Rn. 5 ff.; MüKoStGB/Schmitz Rn. 6; NK-StGB/Kindhäuser/Hoven Rn. 5 alle mwN). Diese Würdigung erfasst nach der hM auch Gesichtspunkte, die in den Regelbeispielen nicht ausdr. erfasst sind, zB die verminderte Schuldfähigkeit (BGHSt 23, 254 (257); BGH NStZ-RR 2003, 297; BeckRS 2024, 5960; zum Entfallen der Regelwirkung bei versuchter Begehung BGH NStZ-RR 2009, 175). Ist die Indizwirkung widerlegt, ist die Strafe aus dem Strafrahmen des § 242 zu entnehmen. Dieser atypische Fall ist vom Gericht in den Urteilsgründen eing. zu begründen (§ 267 Abs. 3 S. 3 StPO; Mitsch StrafR BT II S. 83).\n3 Umgekehrt hat die Regelwirkung auch eine gegenindizierende Komponente (" Gegenschlusswirkung "); dh, wenn kein Regelbeispiel vorliegt, liegt idR auch kein bes. schwerer Fall vor und die Strafe ist, ohne dass dies vom Richter näher begr. werden müsste, dem Strafrahmen des § 242 zu entnehmen (Mitsch StrafR BT II S. 83 f.). Diese kann jedoch, da die Regelbeispiele nicht abschl. sind, zugunsten der Annahme eines unbenannten bes. schweren Falles widerlegt werden, was nach hM wiederum anhand einer Gesamtabwägung zu entscheiden (BGHSt 23, 254 (257); abl. SK-StGB/Hoyer Rn. 10; MüKoStGB/Schmitz Rn. 7) und vom Richter zu begründen ist (§ 267 Abs. 3 S. 3 StPO; Mitsch StrafR BT II S. 84). Wegen der Unbestimmtheit der Vorschr. ist jedenfalls eine restriktive Auslegung notwendig (MüKoStGB/Schmitz Rn. 7). Unrecht und Schuld müssen sich dabei deutlich vom Normalfall des einfachen Diebstahls abheben, um die Anwendung des Ausnahmestrafrahmens zu rechtfertigen (OLG Düsseldorf NJW 2000, 158 (159); Rengier StrafR BT I § 3 Rn. 4). → Rn. 25.</text>\n</GermanCommentary>\n<GermanCommentary>\n  <id_for_citations>PB99UmXm</id_for_citations>\n  <title>Wittig - BeckOK StGB | StGB § 243 Rn. 33-36</title>\n  <text>Rechtsnormen: § 243 StGB\nRechtsgebiete: StrafR, allgemein, HaWiR, ArbSozR, ZivilR, SteuR, OeR\nH. Konkurrenzen\n33 Eigene Konkurrenzen des § 243 mit anderen Vorschr. gibt es wegen der Rechtsnatur des § 243 als Strafzumessungsregel nicht (Fischer/Fischer Rn. 30). Auch § 242 und § 243 bilden ein einheitliches Delikt (NK-StGB/Kindhäuser/Hoven Rn. 61). Nach BGH NStZ 2001, 642 (643 f.) (vgl. hierzu Kargl/Rüdiger NStZ 2002, 202; Sternberg-Lieben JZ 2002, 514) soll deshalb bei einem Einbruchsdiebstahl §§ 242, 243 Abs. 1 S. 2 Nr. 2 nicht die damit regelmäßig einhergehende Sachbeschädigung gem. § 303 konsumiert werden ("Konsumtionslösung"), sondern es soll Idealkonkurrenz (§ 52) vorliegen (zur Problematik Werkmeister JA 2013, 902 (909); WHS StrafR BT II Rn. 254 mwN). Dies sollte nach BGH NStZ-RR 2017, 340 jedoch nur gelten, wenn die Sachbeschädigung bei konkreter Betrachtung von dem regelmäßigen Ablauf eines Einbruchdiebstahls abweicht, von einem eigenständigen Unrechtsgehalt geprägt ist und sich nicht als typische Begleittat erweist. Dies sei insbes. dann der Fall, wenn der Schaden durch Sachbeschädigung über denjenigen durch Diebstahl deutlich hinausgeht.\n33.1 Diese Einschränkung beabsichtigte der 2. Strafsenat (Anfragebeschluss BGH NStZ 2018, 708 mAnm Fahl JR 2019, 107) aufzugeben und stets Tateinheit zwischen Sachbeschädigung und vollendetem Einbruchsdiebstahl anzunehmen. Nachdem sich die übrigen Strafsenate der Ansicht des 2. Strafsenats angeschlossen und etwaig entgegenstehende Rspr. aufgegeben haben (BGH BeckRS 2018, 25767; 2018, 33774; 2018, 25114; 2018, 33775; 2022, 23383) hat der 2. Strafsenat m. Beschl. v. 27.11.2018 den Rechtsprechungswandel vollzogen (BGH NJW 2019, 1086 mAnm Mitsch u. Jäger JA 2019, 386; s. auch BGH BeckRS 2018, 38781; 2022, 23383).\n34 Gleiches müsste dann folgerichtig auch im Verhältnis zwischen §§ 242, 243 Abs. 2 S. 2 Nr. 1 und dem damit regelmäßig einhergehenden Hausfriedensbruch gem. § 123 gelten, wo die hM bish. (zutr.) nicht § 52, sondern Konsumtion annimmt (Fischer/Fischer Rn. 30 mwN). Dem BGH ist allenfalls in Bezug auf § 123 Abs. 1 Var. 2 in besonderen Fallkonstellationen zuzustimmen (vgl. BGH BeckRS 2014, 03301).\n35 Sind durch eine Tat mehrere Regelbeispiele erfüllt, liegt dennoch nur ein Diebstahl in einem bes. schweren Fall vor (MüKoStGB/Schmitz Rn. 92 mwN). Eine Handlungseinheit wird nicht allein dadurch hergestellt, dass der Täter mehrere Diebstähle gewerbsmäßig begeht (LK-StGB/Vogel/Brodowski Rn. 81). Tateinheit ist indes anzunehmen, wenn in ein Gebäude mit einzelnen abgeschlossenen Räumen eingebrochen wird, um sodann in diese Räume einzubrechen (vgl. BGH NStZ-RR 2018, 203). Ansonsten gelten die Ausführungen zu den Konkurrenzen bei § 242 (→ § 242 Rn. 51) auch in den Fällen des § 243. Zum Konkurrenzverhältnis zwischen §§ 242, 243 und § 244 → § 244 Rn. 29.\n36 Zwischen mehreren Regelbeispielen ist Wahlfeststellung möglich (MüKoStGB/Schmitz Rn. 95). Eine wahldeutige Verurteilung ist zwischen dem gewerbsmäßigen Diebstahl nach §§ 242 iVm 243 Abs. 1 S. 2 Nr. 3 und der gewerbsmäßigen Hehlerei nach § 260 Abs. 1 Nr. 1 möglich (BGH NStZ-RR 2018, 47 (49); BeckRS 2013, 00509).\nI. Rechtsfolgen\n37 Bei Vorliegen eines bes. schweren Falls wird der Diebstahl mit einer Freiheitsstrafe von drei Monaten bis zu zehn Jahren bestraft. In der Praxis ist auch in einem bes. schweren Fall Geldstrafe die Regelstrafe (LK-StGB/Vogel/Brodowski Rn. 79). Eine Einstellung gem. §§ 153, 153a StPO bleibt möglich (LK-StGB/Vogel/Brodowski Rn. 80).</text>\n</GermanCommentary>\n<GermanCommentary>\n  <id_for_citations>jsW8jLrN</id_for_citations>\n  <title>Wittig - BeckOK StGB | StGB § 243 Rn. 1-38</title>\n  <text>Rechtsnormen: § 243 StGB\nRechtsgebiete: StrafR, allgemein, HaWiR, ArbSozR, ZivilR, SteuR, OeR\n§ 243 normiert nach der Regelbeispielmethode bes. schwere Fälle des Diebstahls (§ 242), bei deren Erfüllung "in der Regel" (dh nicht nur u. auch nicht stets) der verschärfte Strafrahmen des § 243 anzuwenden ist. Die Regelbeispiele sind keine Tatbestände, sondern (tatbestandsähnliche) ges. Strafzumessungsregeln. Zu den einzelnen Regelbeispielen gibt es eine umfangreiche Kasuistik. Abs. 1 S. 2 Nr. 1 erfasst Fälle, in denen der Täter – wie zB beim Einbruchsdiebstahl – in einen umschlossenen Raum in einer bestimmten Art und Weise zur Ausübung eines Diebstahls eindringt. Abs. 1 S. 2 Nr. 2 regelt Fälle, in denen der Täter durch besondere Schutzvorrichtungen gesicherte Sachen stiehlt. Weitere Regelbeispiele sind der gewerbsmäßige Diebstahl (Abs. 1 S. 2 Nr. 3), der Kirchendiebstahl (Abs. 1 S. 2 Nr. 4), der gemeinschädliche Diebstahl (Abs. 1 S. 2 Nr. 5), der Diebstahl unter Ausnutzung von Notlagen (Abs. 1 S. 2 Nr. 6) und der Diebstahl von Schusswaffen und Sprengstoff (Abs. 1 S. 2 Nr. 7). Der Vorsatz muss sich auf die Verwirklichung der Voraussetzungen der Regelbeispiele beziehen. Abs. 2 enthält für die Regelbeispiele des Abs. 1 S. 2 Nr. 1 – 6 und für unbenannte bes. schwere Fälle eine zwingende Ausschlussklausel, wenn sich der Diebstahl objektiv und subjektiv auf eine geringwertige Sache bezieht. Der Versuch eines Regelbeispiels wirft besondere Probleme auf.</text>\n</GermanCommentary>\n<GermanCommentary>\n  <id_for_citations>ZTWthR2E</id_for_citations>\n  <title>Wittig - BeckOK StGB | StGB § 243 Rn. 14</title>\n  <text>Rechtsnormen: § 243 StGB\nRechtsgebiete: StrafR, allgemein, HaWiR, ArbSozR, ZivilR, SteuR, OeR\n7. Zur Ausführung des Diebstahls\n14 Der Täter muss die Tathandlung der Nr. 1 zur Ausführung des Diebstahls begehen. Das bedeutet, dass der Diebstahlsvorsatz bereits zum Zeitpunkt der Tathandlung (Einbrechen, Einsteigen etc) vorgelegen haben muss (TK-StGB/Bosch Rn. 17; Fischer/Fischer Rn. 11). Auch beim Verweildiebstahl (→ Rn. 13) reicht es nach vorzugswürdiger Ansicht nicht aus, wenn der Täter den Diebstahlsvorsatz erst nach Beginn des Sich-Verbergens fasst (TK-StGB/Bosch Rn. 17). Nicht erforderlich ist nach hM, dass aus dem Raum gestohlen wird, in den der Täter eindringt, es reicht aus, wenn durch die Handlung Hindernisse beseitigt werden, die dem Diebstahl entgegenstehen oder ihn erschweren (LKH/Heger Rn. 8). Auch beim Verweildiebstahl muss nicht aus dem Raum gestohlen werden, in dem sich der Täter verbirgt, er muss aber im selben Gebäude sein (Küper/Zopfs StrafR BT Rn. 474).</text>\n</GermanCommentary>\n<GermanCommentary>\n  <id_for_citations>ApRhE2kd</id_for_citations>\n  <title>Wittig - BeckOK StGB | StGB § 243 Rn. 11-12</title>\n  <text>Rechtsnormen: § 243 StGB\nRechtsgebiete: StrafR, allgemein, HaWiR, ArbSozR, ZivilR, SteuR, OeR\n5. Nachschlüsseldiebstahl\n11 Eindringen mit einem falschen Schlüssel oder einem nicht zur ordnungsgemäßen Öffnung bestimmten Werkzeug ("Nachschlüsseldiebstahl") liegt vor, wenn der Täter einen Verschluss, der nur mit dem richtigen Schlüssel oder einem anderen dafür bestimmten Werkzeug geöffnet werden soll, unter Verwendung des falschen Werkzeugs öffnet und auf diese Weise in den umschlossenen Raum eindringt (vgl. BGH NStZ-RR 2023, 78). Eindringen stellt jedes Betreten eines umschlossenen Raums ohne oder gegen den Willen des Berechtigten dar (MüKoStGB/Schmitz Rn. 25; vgl. zum Eindringen → § 123 Rn. 13). Schlüssel sind neben den gängigen mechanischen Instrumenten aus Metall oder Holz auch elektronische Kunststoffkarten (wie zB in Hotels) oder Transponder und Fernbedienungen (TK-StGB/Bosch Rn. 14; MüKoStGB/Schmitz Rn. 27; BayObLG NJW 1987, 665 (666); zu Funkschlüsseln (sog. Keyless-Go-System) BGH BeckRS 2022, 8071; F. Knauer ZStW 2024 (136), 271 (289)). Falsch ist jeder Schlüssel, der zur Tatzeit vom Berechtigten entweder überhaupt nicht, noch nicht oder nicht mehr zum Öffnen des betreffenden Verschlusses bestimmt ("gewidmet") ist (BGH NStZ 2021, 167 mAnm Kulhanek u. Kudlich JA 2021, 255 zu § 244 Abs. 1 Nr. 3; BGHSt 14, 291 (292); OLG Hamm NJW 1982, 777; HK-StrafR/Duttge Rn. 19; TK-StGB/Bosch Rn. 14 mwN). Der Gebrauch eines falschen ist nach stRspr vom Missbrauch eines richtigen Schlüssels zu unterscheiden, zB wenn dieser vertragswidrig oder unbefugt verwendet wird oder in unbefugte Hände gerät (BGH NStZ 2021, 167 mAnm Kulhanek u. Kudlich JA 2021, 255; Hecker JuS 2021, 370 zu § 244 Abs. 1 Nr. 3 mwN; WHS StrafR BT II Rn. 236; Hecker JuS 2022, 275). Berechtigter ist der, dem die Verfügungsgewalt über den Raum zusteht. Das muss nicht der Eigentümer, sondern kann zB auch der Wohnungsmieter sein (BGH NStZ 2022, 408 mAnm Hecker JuS 2022, 275; BGHSt 13, 15 (16); → Rn. 11.2). Die Bestimmung zur ordnungsgemäßen Öffnung kann von vorneherein befristet (zB hins. des an den Mieter überlassenen Schlüssels; s. aber MüKoStGB/Schmitz Rn. 28; SSW StGB/Kudlich Rn. 14) oder an Bedingungen geknüpft sein. Sie kann auch wieder erlöschen; erforderlich ist zwar keine ausdr. Willenserklärung, aber eine nach außen hervorgetretene Willensentschließung des Berechtigten (Entwidmung; BGH NStZ 2021, 167 mAnm Kulhanek u. Kudlich JA 2021, 255; Hecker JuS 2021, 370 zu § 244 Abs. 1 Nr. 3; BGHSt 13, 15). Ohne Entwidmung handelt es sich um den Missbrauch eines richtigen Schlüssels, der nicht von Abs. 1 Nr. 1 erfasst wird (OLG Köln StraFo 2010, 300). Bei Benutzung eines heimlich entwendeten Haus- und Wohnungsschlüssels ist aber ein unbenannter bes. schwerer Fall (→ Rn. 25) des Diebstahls gem. § 243 in Betracht zu ziehen (BGH BeckRS 1986, 31109793; NStZ-RR 2023, 390 (391)).\n11.1 Ein für Notfälle vorhandener Schlüssel oder ein Reserveschlüssel ist bereits gewidmet und damit idR nicht falsch (NK-StGB/Kindhäuser/Hoven Rn. 16; MüKoStGB/Schmitz Rn. 28 jew. mwN); anders aber, wenn ein solcher Schlüssel durch den Berechtigten noch nicht zur Verwendung bestimmt wurde (MüKoStGB/Schmitz Rn. 28). Das bloße Verlegen oder der sonstige Verlust, auch durch Diebstahl, reicht für eine Entwidmung nicht ohne weiteres aus; wird der Diebstahl oder das sonstige Abhandenkommen entdeckt, ist idR auch ohne einen ausdr. oder konkludenten Entwidmungsakt von einer Entwidmung auszugehen (BGH NStZ 2021, 167 mAnm Kulhanek u. Kudlich JA 2021, 255; Hecker JuS 2021, 370 zu § 244 Abs. 1 Nr. 3; BGHSt 21, 189; BGH StV 1993, 422; Rengier StrafR BT I § 3 Rn. 16). Ein bei dem Berechtigten in Vergessenheit geratener Schlüssel ist kein falscher Schlüssel (BGH NStZ 2021, 167 mAnm Kulhanek u. Kudlich JA 2021, 255; Hecker JuS 2021, 370 zu § 244 Abs. 1 Nr. 3); ebenso wenig reicht es für die Annahme einer Entwidmung aus, wenn ein Reserveschlüssel gebraucht oder ein neuer Schlüssel angefertigt wird (MüKoStGB/Schmitz Rn. 28; aA RGSt 52, 84; NK-StGB/Kindhäuser/Hoven Rn. 16). Dies gilt zB auch bei unbefugter Benutzung einer einem anderen abhanden gekommenen Bankautomatenkarte (TK-StGB/Bosch Rn. 14; BayObLG NJW 1987, 663 (664); gegen Schlüsselfunktion von "Berechtigungsnachweisen" Matt/Renzikowski/Schmidt Rn. 8). Ein von Dritten heimlich angefertigter Schlüssel, von dem der Berechtigte nichts weiß, ist dagegen kein richtiger Schlüssel (Fischer/Fischer Rn. 8a; aA KG StV 2004, 544).\n11.2 Behält ein Vermieter einen Schlüssel ohne Wissen des Mieters zurück, wird dieser durch die Vermietung der Wohnung entwidmet und damit "falsch" (BGH NStZ 2022, 408 (409) mAnm Hecker JuS 2022, 275; vgl. RGSt 11, 436 (437 f.)). Gleiches gilt, wenn der Vermieter selbst keine Kenntnis mehr von der Existenz eines weiteren Wohnungsschlüssels hat (BGH NStZ 2022, 408 (409) mAnm Hecker JuS 2022, 275). Durch die Vermietung einer Wohnung einschließl. der Übergabe der Wohnungsschlüssel wird zumindest konkludent zum Ausdruck gebracht, dass nunmehr nur noch die im Besitz des Mieters befindlichen Schlüssel zur ordnungsgemäßen Öffnung der Wohnung bestimmt sind, außer der Vermieter behält mit Kenntnis des Mieters vereinbarungsgemäß einen weiteren Schlüssel (BGH NStZ 2022, 408 (409) mAnm Hecker JuS 2022, 275).\n12 Andere nicht zur ordnungsgemäßen Öffnung bestimmte Werkzeuge sind solche, durch die der Verschlussmechanismus ordnungswidrig betätigt wird (RGSt 53, 277), zB Dietriche, Schraubenzieher, Magnete, Zangen, Pickwerkzeuge, Drähte und Haken, auch elektronische Instrumente zum Ersatz elektronischer Schlüssel, nicht aber Stemmeisen und Brechwerkzeuge (MüKoStGB/Schmitz Rn. 30; NK-StGB/Kindhäuser/Hoven Rn. 17; TK-StGB/Bosch Rn. 15 jew. mwN). Auch das Öffnen mit dem Finger erfüllt das Regelbeispiel nicht (BGH NJW 1956, 271). Wird der Verschluss aufgebrochen, ist die Variante des Einbrechens (→ Rn. 8) erfüllt (BGH NJW 1956, 271; Eisele StrafR BT II Rn. 117). Nicht erfasst sind Instrumente, mit denen eine Verriegelung nicht geöffnet, sondern die Verriegelung verhindert wird, so zB Störsender, die den per Funkfernbedienung ausgelösten Schließmechanismus eines Kfz manipulieren (BGH NStZ 2018, 212 mAnm Hoven; F. Knauer ZStW 2024 (136), 271 (289)). In diesen Fällen wird aber regelmäßig ein unbenannter bes. schwerer Fall vorliegen (→ Rn. 25).</text>\n</GermanCommentary>\n<GermanCommentary>\n  <id_for_citations>N05OADyP</id_for_citations>\n  <title>Wittig - BeckOK StGB | StGB § 243 Rn. 1-38</title>\n  <text>Rechtsnormen: § 243 StGB\nRechtsgebiete: StrafR, allgemein, HaWiR, ArbSozR, ZivilR, SteuR, OeR\n§ 243 Besonders schwerer Fall des Diebstahls\n(1) 1 In besonders schweren Fällen wird der Diebstahl mit Freiheitsstrafe von drei Monaten bis zu zehn Jahren bestraft. 2 Ein besonders schwerer Fall liegt in der Regel vor, wenn der Täter\n1. zur Ausführung der Tat in ein Gebäude, einen Dienst- oder Geschäftsraum oder in einen anderen umschlossenen Raum einbricht, einsteigt, mit einem falschen Schlüssel oder einem anderen nicht zur ordnungsmäßigen Öffnung bestimmten Werkzeug eindringt oder sich in dem Raum verborgen hält,\n2. eine Sache stiehlt, die durch ein verschlossenes Behältnis oder eine andere Schutzvorrichtung gegen Wegnahme besonders gesichert ist,\n3. gewerbsmäßig stiehlt,\n4. aus einer Kirche oder einem anderen der Religionsausübung dienenden Gebäude oder Raum eine Sache stiehlt, die dem Gottesdienst gewidmet ist oder der religiösen Verehrung dient,\n5. eine Sache von Bedeutung für Wissenschaft, Kunst oder Geschichte oder für die technische Entwicklung stiehlt, die sich in einer allgemein zugänglichen Sammlung befindet oder öffentlich ausgestellt ist,\n6. stiehlt, indem er die Hilflosigkeit einer anderen Person, einen Unglücksfall oder eine gemeine Gefahr ausnutzt oder\n7. eine Handfeuerwaffe, zu deren Erwerb es nach dem Waffengesetz der Erlaubnis bedarf, ein Maschinengewehr, eine Maschinenpistole, ein voll- oder halbautomatisches Gewehr oder eine Sprengstoff enthaltende Kriegswaffe im Sinne des Kriegswaffenkontrollgesetzes oder Sprengstoff stiehlt.\n(2) In den Fällen des Absatzes\u2004 1 Satz\u20042 Nr. 1 bis 6 ist ein besonders schwerer Fall ausgeschlossen, wenn sich die Tat auf eine geringwertige Sache bezieht.</text>\n</GermanCommentary>\n<GermanCommentary>\n  <id_for_citations>iH1WVjvL</id_for_citations>\n  <title>Wittig - BeckOK StGB | StGB § 243 Rn. 16, 16.1</title>\n  <text>Rechtsnormen: § 243 StGB\nRechtsgebiete: StrafR, allgemein, HaWiR, ArbSozR, ZivilR, SteuR, OeR\n2. Verschlossenes Behältnis\n16 Als Beispiel einer Schutzvorrichtung nennt Nr. 2 das verschlossene Behältnis. Behältnis ist ein zur Aufnahme von Sachen dienendes und sie umschließendes Raumgebilde, das – im Gegensatz zum umschlossenen Raum (Nr. 1, → Rn. 5) – nicht dazu bestimmt ist, von Menschen betreten zu werden (BGHSt 1, 158 (163); HK-StrafR/Duttge Rn. 26; MüKoStGB/Schmitz Rn. 33 mwN). Verschlossen ist ein Behältnis, wenn sein Inhalt durch ein Schloss, eine andere technische Schließvorrichtung oder in sonstiger Weise (zB durch Zukleben, festes Verschnüren, Zunageln) gegen den ordnungswidrigen Zugriff von außen bes. gesichert ist (MüKoStGB/Schmitz Rn. 33 mwN; Küper/Zopfs StrafR BT Rn. 99). Behältnisse können beweglich oder unbeweglich sein.\n16.1 Bsp.: (WHS StrafR BT II Rn. 243; MüKoStGB/Schmitz Rn. 33 jew. mwN) sind der Kofferraum eines Kfz (BGHSt 13, 81 (82)), Kisten, Koffer, verschließbare Möbel (RGSt 30, 390), Schaukästen (BGHSt 15, 134), Kassetten, Aktentaschen, Sammelbüchsen, Briefkästen, Münzgaszähler, Automaten (BGHSt 9, 173), geschlossene oder codierte Registrierkassen (AG Freiburg NJW 1994, 400; LTZ/Kretschmer Rn. 13). Ein verschlossenes Behältnis soll gegen Wegnahme auch dann noch bes. gesichert sein, wenn der Täter dieses mit dem zuvor aufgefundenen oder entwendeten Schlüssel öffnet, sofern er unbefugt handelt (BGH NJW 2010, 3175; OLG Karlsruhe NStZ-RR 2010, 48 mAnm Bosch JA 2009, 905; KG NJW 2012, 1093 mAnm Bachmann/Goeck ZJS 2012, 279; Jahn Jus 2012, 468). Ungeklärt durch die Rspr. ist allerdings noch, ob das Regelbeispiel in diesen Fällen auch dann erfüllt ist, wenn der Schlüssel im Schloss steckt oder direkt daneben liegt (→ Rn. 18.1). Der Täter soll auch dann eine durch ein verschlossenes Behältnis bes. gesicherte Sache stehlen, wenn er als Unberechtigter durch Täuschung einen gutgläubigen Dritten dazu bewegt, den zur ordnungsgemäßen Öffnung bestimmten Zugangscode für den Schließmechanismus zu verwenden (KG NJW 2012, 1093 (1094)). Bei Tresoren und Containern, die entspr. ihrer Größe auch von Menschen betreten werden können, handelt es sich um Räumlichkeiten iSd Nr. 1 (→ Rn. 5), sonst um verschlossene Behältnisse (MüKoStGB/Schmitz Rn. 33).</text>\n</GermanCommentary>\n<GermanCommentary>\n  <id_for_citations>cxpPjOjD</id_for_citations>\n  <title>Wittig - BeckOK StGB | StGB § 243 Rn. 17, 17.1</title>\n  <text>Rechtsnormen: § 243 StGB\nRechtsgebiete: StrafR, allgemein, HaWiR, ArbSozR, ZivilR, SteuR, OeR\n3. Andere Schutzvorrichtung\n17 Andere Schutzvorrichtungen sind alle sonstigen besonderen Vorrichtungen, die geeignet oder dazu bestimmt sind, die Wegnahme einer Sache wenigstens zu erschweren, ohne sie – in Abgrenzung zum verschlossenen Behältnis – zu umhüllen (BayObLG JZ 1973, 324; Fischer/Fischer Rn. 15; MüKoStGB/Schmitz Rn. 34; SK-StGB/Hoyer Rn. 29).\n17.1 Bsp.: (LTZ/Kretschmer Rn. 14; Fischer/Fischer Rn. 15; MüKoStGB/Schmitz Rn. 34) sind Alarmanlagen, Lenkradschlösser, elektronische Wegfahrsperren, Fahrradschlösser, Sicherungsketten und Sicherungen an Waren, die das Wegnehmen verhindern sollen, zB Sicherungskabel an elektronischen Geräten (Fischer/Fischer Rn. 15; → Rn. 18.2). Nach Rengier StrafR BT I § 3 Rn. 24; Berkl JA 2006, 280 soll auch das Zündschloss eine Schutzvorrichtung sein. Keine Schutzvorrichtungen sind Software-Zugangssperren für einen Programmablauf (zB PIN bei Bankautomaten); zum sog. Bankomatenmissbrauch vgl. TK-StGB/Bosch Rn. 25 mwN. Auch Vorrichtungen, die lediglich die Nutzung der weggenommenen Sache verhindern sollen, wie zB elektronische Codesicherungen an (Auto)Radios, stellen keine Schutzvorrichtung dar (MüKoStGB/Schmitz Rn. 34 mwN).</text>\n</GermanCommentary>\n<GermanCommentary>\n  <id_for_citations>ABEbav06</id_for_citations>\n  <title>Wittig - BeckOK StGB | StGB § 243 Rn. 4-24</title>\n  <text>Rechtsnormen: § 243 StGB\nRechtsgebiete: StrafR, allgemein, HaWiR, ArbSozR, ZivilR, SteuR, OeR\nB. Regelbeispiele\nI. Einbruchs-, Einsteige-, Nachschlüssel- und Verweildiebstahl (Abs. 1 S. 2 Nr. 1)\n1. Grundlagen\n4 Abs. 1 S. 2 Nr. 1 erfasst Fälle, in denen der Täter in einen umschlossenen Raum in einer bestimmten Art und Weise zur Ausübung eines Diebstahls eindringt. Der Unrechtsgehalt wird durch diese Begehungsweisen (Einbrechen, Einsteigen etc) erhöht, weil sie die an sich schon durch den Gewahrsamsbruch bewirkte Friedensstörung noch steigern (BayObLG NJW 1973, 1205; MüKoStGB/Schmitz Rn. 10 mwN).</text>\n</GermanCommentary>\n<GermanCommentary>\n  <id_for_citations>lDCF35yI</id_for_citations>\n  <title>Wittig - BeckOK StGB | StGB § 243 Rn. 8-9</title>\n  <text>Rechtsnormen: § 243 StGB\nRechtsgebiete: StrafR, allgemein, HaWiR, ArbSozR, ZivilR, SteuR, OeR\n3. Einbrechen\n8 Einbrechen ist das gewaltsame – mit nicht ganz unerheblicher Kraftentfaltung oder entspr. Werkzeugeinsatz verbundene (BGH NStZ 2000, 143) – nicht notwendigerweise substanzverletzende (RGSt 4, 353 (354); TK-StGB/Bosch Rn. 11) Öffnen oder Erweitern einer dem Zutritt entgegenstehenden Umschließung durch Schaffung eines Zugangs oder einer Zugriffsmöglichkeit von außen (BGH BeckRS 2014, 6840; TK-StGB/Bosch Rn. 11; Küper/Zopfs StrafR BT Rn. 189).\n8.1 Die erforderliche Kraftentfaltung fehlt zB beim einfachen Hochheben und Beseitigen eines beweglichen Zauns (BGH NStZ 2000, 143 (144)), bei einfachem Lösen der Arretierung eines verschlossenen Gartentors (OLG Karlsruhe NStZ-RR 2005, 140), bei der Entriegelung eines offen stehenden Kippfensters (WHS StrafR BT II Rn. 234) oder beim Durchgreifen durch einen Türspalt zum Zwecke der Entriegelung (LKH/Heger Rn. 10), nicht aber beim Ausheben von Türen (TK-StGB/Bosch Rn. 11) oder beim Auseinanderbiegen der Flügel eines Scheunentors (RGSt 4, 353 (354)). Ob das Aufdrücken eines nicht verriegelten Seitenfensters eines Kfz zum Zwecke der Entriegelung reicht (BGH NJW 1956, 389), ist mehr als fraglich (KHH StrafR BT II Rn. 137; LKH/Heger Rn. 10; SSW StGB/Kudlich Rn. 11 mwN).\n9 Das Eintreten des Täters in den gewaltsam geöffneten umschlossenen Raum ist nicht erforderlich, es reicht, wenn er mit der Hand hineinlangt oder sich eines Werkzeugs bedient, um die Sache herauszuholen (RGSt 56, 48; BGH NStZ 1985, 217 (218); MüKoStGB/Schmitz Rn. 21 mwN). Stehlen aus dem umschlossenen Raum ist nicht erforderlich, es reicht auch das Einbrechen, um den umschlossenen Raum als Ganzes wegzunehmen (zB bei Diebstahl eines Kfz; LKH/Heger Rn. 10; NK-StGB/Kindhäuser/Hoven Rn. 13). Kein Einbrechen ist die gewaltsame Beseitigung der Umschließung von innen, zB zur Flucht oder zum Fortschaffen der Beute (RGSt 55, 210 (211); LKH/Heger Rn. 10 mwN).</text>\n</GermanCommentary>\n<GermanCommentary>\n  <id_for_citations>OeD0FULO</id_for_citations>\n  <title>Wittig - BeckOK StGB | StGB § 243 Rn. 15-19</title>\n  <text>Rechtsnormen: § 243 StGB\nRechtsgebiete: StrafR, allgemein, HaWiR, ArbSozR, ZivilR, SteuR, OeR\nII. Diebstahl einer durch eine Schutzvorrichtung bes. gesicherten Sache (Abs. 1 S. 2 Nr. 2)\n1. Grundlagen\n15 Abs. 1 S. 2 Nr. 2 regelt den Diebstahl von Sachen, die durch ein verschlossenes Behältnis oder eine andere Schutzvorrichtung gegen Wegnahme bes. gesichert sind ("Aufbruchdiebstahl"). Abs. 1 S. 2 Nr. 1 und Abs. 1 S. 2 Nr. 2 schließen sich bzgl. ein und desselben Raumgebildes gegenseitig aus, nicht aber bzgl. verschiedener Barrieren (Rengier StrafR BT I § 3 Rn. 21 f.).</text>\n</GermanCommentary>',
               'name': 'german_commentary_search',
               'reasoning_content': None,
               'role': 'tool',
               'tool_call_id': 'call_68edba1c42a846fba6affee0',
               'tool_calls': None},
              {'content': '\n\n',
               'name': None,
               'reasoning_content': 'Die Kommentare geben schon gute Hinweise, aber ich sollte auch noch nach Gerichtsentscheidungen suchen, die sich speziell mit den Anforderungen an die Überwindung einer besonderen Sicherung befassen, um die konkreten Anforderungen an die Tätigkeit des Täters besser zu verstehen.\n\nDie Kommentare haben bereits einige wichtige Punkte geliefert:\n1. Es kommt nicht auf die Art und Weise an, wie der Täter die Schutzvorrichtung beseitigt - ob und wie die Schutzvorrichtung beseitigt wird, ist irrelevant.\n2. Die Sicherung muss eine "besondere Sicherung gegen Wegnahme" darstellen und die Wegnahme "wesentlich erschweren".\n3. Bei kleinen Behältnissen ist fraglich, ob diese überhaupt die Sache vor Wegnahme schützen können, wenn sie mit samt der Sicherung entwendet werden.\n4. Bei Spielautomaten liegt Nr. 2 nur vor, wenn die Sicherungsvorrichtung überwunden wird (durch Aufbrechen oder regelwidriger Manipulation von außen), nicht aber durch Überlistung.\n\nIch sollte jetzt noch nach Gerichtsentscheidungen suchen, um die Anforderungen an die Überwindungstätigkeit weiter zu konkretisieren.',
               'role': 'assistant',
               'tool_call_id': None,
               'tool_calls': [{'function': {'arguments': {'court_instances': None,
                                                          'laws': ['stgb 243'],
                                                          'list_of_laws': None,
                                                          'query': 'Überwindung besonderer Sicherung § 243 Abs 1 Nr 2 StGB Anforderungen Täter Tätigkeit'},
                                            'name': 'german_court_decision_search'},
                               'id': 'call_de611e9ce69046018d5ff2db',
                               'type': 'function'}]},
              {'content': '<GermanCourtDecision>\n  <id_for_citations>tou5XNY9</id_for_citations>\n  <title>BGH 2 StR 385/10</title>\n  <text>Verkündet am: 2010-08-05\nRechtsgebiete: StPR, StR\n4 Dies entspricht dem Wortlaut und dem Zweck des § 243 Abs. 1 Satz 2 Nr. 2 StGB. Dient das Behältnis nach seiner erkennbaren Zweckbestimmung wenigstens unter anderem auch zur Sicherung der darin aufbewahrten Sache gegen Diebstahl, wie es bei einem Tresor idealtypisch der Fall ist, dann ist das verschlossene Behältnis ein Spezialfall einer Schutzvorrichtung im Sinne der Vorschrift. Das Regelbeispiel setzt voraus, dass das Behältnis verschlossen ist. Weitere Sicherungen, etwa durch Wegschließen des Schlüssels, sind danach zu seiner Erfüllung nicht mehr erforderlich. Der Täter muss - sofern er nicht sogar die Sache mitsamt dem Behältnis stiehlt - die Sicherung überwinden, wobei es aber nicht darauf ankommt, wie er das bewirkt (vgl. BT-Drucks. IV/650 S. 403; Fischer, StGB, 57. Aufl., § 243 Rn. 17). § 243 Abs. 1 Satz 2 Nr. 2 StGB betont nämlich die besondere Sicherung des Diebstahlsobjekts, während § 243 Abs. 1 Satz 2 Nr. 1 StGB besondere Arten der Tatausführung bei einer allgemeinen Sicherung des Gegenstands hervorhebt; auf eine besondere Gestaltung der Tathandlung über das Überwinden der Sicherung hinaus kommt es dagegen bei § 243 Abs. 1 Satz 2 Nr. 2 StGB nicht an (vgl. OLG Frankfurt, NJW 1988, 3028). Daher scheidet die Anwendung des Regelbeispiels für einen besonders schweren Fall des Diebstahls wegen der Wegnahme einer Sache aus einem verschlossenen Behältnis auch dann nicht aus, wenn der Verschluss mit dem dafür vorgesehenen Schlüssel geöffnet wird. Allenfalls dann, wenn der Benutzer des Schlüssels zu dessen Verwendung befugt ist, könnte für ihn die Eigenschaft des Behältnisses als besondere Diebstahlssicherung entfallen (vgl. OLG Hamm, JR 1982, 119 mit abl. Anm. Schmid; Schmitz in MünchKomm, StGB, 2003, § 243 Rn. 35). Jedenfalls wenn ein Unbefugter den Schlüssel an sich nimmt und er damit das Behältnis öffnet, überwindet er die Diebstahlssicherung, die sich aus dem Verschlusszustand des Behältnisses ergibt (vgl. OLG Karlsruhe, NStZ-RR 2010, 48; Fischer, StGB, § 243 Rn. 17; LK/Vogel, StGB, 12. Aufl., § 243 Rn. 32).\n5 Die Erfüllung des Regelbeispiels führt grundsätzlich zur Anwendung des Ausnahmestrafrahmens. Das Landgericht hat rechtsfehlerfrei angenommen, dass auch nach den Umständen des konkreten Falles kein Grund dafür besteht, die Regelwirkung hier entfallen zu lassen.\nRissing-van SaanFischerSchmitt\nEschelbach Ott</text>\n</GermanCourtDecision>\n<GermanCourtDecision>\n  <id_for_citations>PTDfeEJT</id_for_citations>\n  <title>OLG Hamm 1 RVs 91/16</title>\n  <text>Verkündet am: 2016-12-27\nRechtsgebiete: StPR, StR\n5 Der Anwendung des nach den §§ 23 Abs. 2, 49 Abs. 1 StGB gemilderten Strafrahmens des § 243 Abs. 1 StGB steht es hier auch nicht entgegen, dass der Angeklagte noch mit keiner der von ihm und seinem Mittäter nach den Feststellungen beabsichtigten Handlungen des Einbrechens, Einsteigens oder des Verborgenhaltens im Sinne des § 243 Abs. 1 Nr. 1 StGB konkret begonnen hatte. Denn die vorgenannten Handlungen zur Überwindung des Zauns waren nach dem festgestellten Tatplan - im Unterschied etwa zu der der Entscheidung BGH NStZ 1995, 339 zugrunde liegenden Konstellation - notwendig und unmittelbar vorgelagerte Zwischenschritte zur Durchführung der Wegnahme, für die - wie bereits ausgeführt - die Strafkammer rechtsfehlerfrei ein unmittelbares Ansetzen bejaht hat. Bei einer solchen Fallgestaltung fällt das unmittelbare Ansetzen hinsichtlich der Voraussetzungen des § 243 Abs. 1 Nr. 1 StGB zumindest in der Regel - und so auch vorliegend - mit dem Anfang des Diebstahlsversuchs zusammen (bzgl. § 243 Abs. 1 Nr. 2 StGB vgl. BayObLG, NStZ 1997, 442; Sander, NStZ 1999, 36; Kudlich/Schur in: Satzger/Schluckebier/Widmaier, StGB, 2. Aufl., § 22 Rn. 45 f.; LK-Vogel, StGB, 12. Aufl., § 243 Rn. 72, 74; krit. Graul, JuS 1999, 852; Wolters, JR 1999, 37; Duttge in: Dölling/Duttge/Rössner, Gesamtes Strafrecht, 3. Aufl., § 243 Rn. 63; Eser/Bosch in: Schönke/Schröder, a.a.O., § 243 Rn. 44; Schmitz in: MK-StGB, 3. Aufl., § 243 Rn. 90).</text>\n</GermanCourtDecision>\n<GermanCourtDecision>\n  <id_for_citations>DTL5LNOq</id_for_citations>\n  <title>BGH 1 StR 79/18</title>\n  <text>Verkündet am: 2018-06-26\nRechtsgebiete: StPR, StR\n10 Der Strafausspruch hält revisionsgerichtlicher Überprüfung nicht stand, weil die getroffenen Feststellungen dem Revisionsgericht keine Prüfung ermöglichen, ob die Strafkammer das Regelbeispiel des § 243 Abs. 1 Satz 2 Nr. 2 StGB ohne Rechtsfehler verneint hat. Nach dieser Vorschrift liegt ein besonders schwerer Fall des Diebstahls in der Regel dann vor, wenn der Täter eine Sache stiehlt, die durch ein verschlossenes Behältnis oder eine andere Schutzvorrichtung gegen Wegnahme besonders gesichert ist. Die Schutzvorrichtung muss tatsächlich funktionsfähig und aktiviert sein. Deshalb ist ein offenes Schloss oder ein geöffneter Tresor keine Schutzvorrichtung gegen Wegnahme (z.B. BGH, Beschluss vom 20. April 2005 – 1 StR 123/05; Fischer, StGB, 65. Aufl., § 243 Rn. 16a; LK-StGB/Vogel, 12. Aufl., § 243 Rn. 29).</text>\n</GermanCourtDecision>\n<GermanCourtDecision>\n  <id_for_citations>KggPuLF4</id_for_citations>\n  <title>BGH 5 StR 15/20</title>\n  <text>Verkündet am: 2020-04-28\nRechtsgebiete: StPR, StR\n7 Nicht erforderlich für das unmittelbare Ansetzen zur geplanten Wegnahme ist, dass der angegriffene Schutzmechanismus auch erfolgreich überwunden wird (vgl. BGH, Urteil vom 7. Februar 1952 - 5 StR 12/52, BGHSt 2, 380; Beschlüsse vom 18. November 1985 - 3 StR 291/85, BGHSt 33, 370, und vom 27. November 2018 - 2 StR 481/17, BGHSt 63, 253, 254; OLG Hamm, MDR 1976, 155). Deshalb reicht der Beginn des Einbrechens, Einsteigens oder Eindringens im Sinne von § 243 Abs. 1 Satz 2 Nr. 1, § 244 Abs. 1 Nr. 3 StGB regelmäßig aus, um einen Versuchsbeginn anzunehmen (vgl. BGH, Urteil vom 8. Februar 1984 - 3 StR 414/83, NStZ 1984, 262 mwN). Soweit die bisherige Rechtsprechung des Senats entgegensteht (vgl. insbesondere Beschlüsse vom 4. Juli 2019 - 5 StR 274/19, und vom 1. August 2019 - 5 StR 185/19, NStZ 2019, 716; hierzu näher Kudlich, NStZ 2020, 34), hält er hieran nicht fest.</text>\n</GermanCourtDecision>\n<GermanCourtDecision>\n  <id_for_citations>sxcT26z0</id_for_citations>\n  <title>OLG Hamm 1 RVs 91/16</title>\n  <text>Verkündet am: 2016-12-27\nRechtsgebiete: StPR, StR\nAmtliche Leitsätze: 1. Abhängig von konkreten Tatumständen kann es für den Schluss auf ein nach den Vorstellungen der Täter anzunehmendes unmittelbares Ansetzen zu einem Diebstahl genügen, dass der Zaun, den die Täter zum Abtransport des bereits im Außenbereich eines Baumarkts deponierten Diebesguts überwinden mussten, zuvor schon häufiger kein entscheidendes Hindernis für solche Diebstähle dargestellt hatte und ein dortiges Hochregal sowohl von einem Mitarbeiter des Mitarbeiters als auch zumindest von einem der früheren Täter ohne Probleme bzw. leichter und ungefährlicher als bei Verwendung einer Leiter erklettert werden kann bzw. konnte. 2. In dieser Konstellation steht der Anwendung des nach den §§ 23 Abs. 2, 49 Abs. 1 StGB gemilderten Strafrahmens des § 243 Abs. 1 StGB nicht entgegen, dass die Täter noch mit keiner der in § 243 Abs. 1 S. 2 Nr. 1 StGB aufgeführten Handlungen konkret begonnen haben, sofern eine dieser Handlungen zur Überwindung des Zauns nach dem Tatplan notwendig und unmittelbar vorgelagerte Zwischenschritte zur Durchführung der Wegnahme waren. Bei einer solchen Fallgestaltung fällt das unmittelbare Ansetzen hinsichtlich der Voraussetzungen des § 243 Abs. 1 Nr. 1 StGB zumindest in der Regel mit dem Anfang des Diebstahlsversuchs zusammen.</text>\n</GermanCourtDecision>\n<GermanCourtDecision>\n  <id_for_citations>mK9riatq</id_for_citations>\n  <title>OLG Dresden 2 OLG 22 Ss 14/15</title>\n  <text>Verkündet am: 2015-03-12\nRechtsgebiete: StPR, StR\n13 Voraussetzung für die indizierte Strafschärfung ist es, dass eine Schutzvorrichtung überwunden wird, welche nach ihrer Zweckbestimmung gerade den Gewahrsam des Berechtigten gegen den Bruch durch einen Unbefugten sichern soll. Danach darf der Gewahrsamsbruch bei Wirksamwerden der Schutzvorrichtung noch nicht vollendet sein. Mit Recht verweist die Revision darauf, dass elektronische Sicherungsetiketten diesen Schutz nicht gewähren. Denn sie dienen nicht dem Schutz gegen einen Gewahrsamsbruch, sondern nur der Wiedererlangung des bereits an den Täter verlorengegangenen Gewahrsams durch Ergreifen des Berechtigten oder seiner Hilfspersonen (OLG Stuttgart NStZ 1985, 385; OLG Frankfurt/Main MDR 1993, 671 [672]; Ruß in LK, StGB, 11. Auflage, § 243 Rdnr. 19; KG Berlin, Beschluss vom 04. Februar 1998 - (4) 1 Ss 209/97 (135/97) -, Rdnr. 7, zitiert nach juris).</text>\n</GermanCourtDecision>\n<GermanCourtDecision>\n  <id_for_citations>U154hS0D</id_for_citations>\n  <title>LG Münster 115 O 265/15</title>\n  <text>Verkündet am: 2016-09-08\nRechtsgebiete: PrivVersR\nRedaktioneller Leitsatz: Die sog. Erweiterte Schlüsselklausel ist ihrem klaren Wortlaut nach nicht als ein Risikoausschluss zu verstehen, sondern enthält im Gegenteil insoweit eine Ergänzung des Versicherungsschutzes, als ein Einbruchdiebstahl als versichertes Risiko bei dieser Begehungsform an sich gar nicht vorliegt. Denn es fehlt das typische Gepräge der übrigen Varianten, bei denen dem Täter die Überwindung besonderer Schutzmaßnahmen abverlangt wird, wie es bei der Bestimmung des § 243 I Nr. 1 StGB der Fall ist.</text>\n</GermanCourtDecision>\n<GermanCourtDecision>\n  <id_for_citations>qLVBKKYo</id_for_citations>\n  <title>BGH 5 StR 15/20</title>\n  <text>Verkündet am: 2020-04-28\nRechtsgebiete: StPR, StR\n6 b) Bei Diebstahlsdelikten ist demgemäß darauf abzustellen, ob aus Tätersicht bereits die konkrete Gefahr eines ungehinderten Zugriffs auf das in Aussicht genommene Stehlgut besteht (vgl. BGH, Beschluss vom 14. März 2001 - 3 StR 48/01, BGHR StGB § 22 Ansetzen 29; siehe auch schon RGSt 54, 254, 255). Hierfür ist entscheidend, ob der Gewahrsam durch Schutzmechanismen gesichert ist (vgl. BGH, Beschluss vom 20. September 2016 - 2 StR 43/16, aaO; OLG Hamm MDR 1976, 155). Ist dies der Fall, reicht für den Versuchsbeginn der erste Angriff auf einen solchen Schutzmechanismus regelmäßig aus, wenn sich der Täter bei dessen Überwindung nach dem Tatplan ohne tatbestandsfremde Zwischenschritte, zeitliche Zäsur oder weitere eigenständige Willensbildung einen ungehinderten Zugriff auf die erwartete Beute vorstellt (vgl. bereits RGSt 53, 217, 218; 54, 35). Sollen mehrere gewahrsamssichernde Schutzmechanismen hintereinander überwunden werden, ist schon beim Angriff auf den ersten davon in der Regel von einem unmittelbaren Ansetzen zur Wegnahme auszugehen, wenn die Überwindung aller Schutzmechanismen in unmittelbarem zeitlichen und räumlichen Zusammenhang mit paraten Mitteln erfolgen soll (vgl. auch BGH, Beschluss vom 8. Februar 2018 - 1 StR 228/17, NStZ-RR 2018, 203: Einbruch in ein Bürogebäude, um anschließend in einzelne Büros einzubrechen). Wird der Schutz des Gewahrsams durch eine hierzu bereite Person gewährleistet, liegt Versuch vor, wenn auf diese (durch Täuschung oder Drohung) mit dem Ziel einer Gewahrsamslockerung eingewirkt wird und die Wegnahme in unmittelbarem zeitlichen und örtlichen Zusammenhang damit erfolgen soll (vgl. BGH, Urteile vom 16. September 2015 - 2 StR 71/15, BGHR StGB § 22 Ansetzen 39, und vom 10. August 2016 - 2 StR 493/15, StV 2017, 441, jeweils mwN: Begehren um Einlass beim Trickdiebstahl aus der Wohnung).</text>\n</GermanCourtDecision>\n<GermanCourtDecision>\n  <id_for_citations>rT3eePX5</id_for_citations>\n  <title>BGH 2 StR 4/25</title>\n  <text>Verkündet am: 2025-05-07\n8 (1) Eine Straftat versucht, wer nach seiner Vorstellung von der Tat zur Verwirklichung des Tatbestandes unmittelbar ansetzt (§ 22 StGB). Erforderlich ist hierfür nicht die Verwirklichung mindestens eines Tatbestandsmerkmals. Genügend ist vielmehr auch ein für sich gesehen noch nicht tatbestandsmäßiges Handeln, soweit es nach der Vorstellung des Täters der Verwirklichung eines Tatbestandsmerkmals räumlich und zeitlich unmittelbar vorgelagert ist oder nach dem Tatplan im ungestörten Fortgang ohne Zwischenakt in die Tatbestandsverwirklichung einmünden soll. Diese abstrakten Maßstäbe bedürfen angesichts der Vielzahl denkbarer Sachverhaltsgestaltungen der wertenden Konkretisierung unter Beachtung der Umstände des Einzelfalls. Maßgeblicher Orientierungspunkt ist dabei angesichts der Fassung des § 22 StGB die Vorstellung des Täters, das heißt der Tatplan, der über die Abgrenzung zwischen Vorbereitung und Versuch entscheidet (st. Rspr.; vgl. etwa BGH, Beschlüsse vom 17. Juli 2018 – 2 StR 123/18, NStZ 2019, 79 Rn. 4 f., und vom 12. März 2023 – 3 StR 422/23, BGHR StGB § 22 Ansetzen 42 Rn. 11, jew. mwN). Ist der Gewahrsam durch einen Schutzmechanismus gesichert, reicht für den Versuchsbeginn der erste Angriff auf diesen regelmäßig aus, wenn sich der Täter bei dessen Überwindung nach dem Tatplan ohne tatbestandsfremde Zwischenschritte, zeitliche Zäsur oder weitere eigenständige Willensbildung einen ungehinderten Zugriff auf die erwartete Beute vorstellt (vgl. BGH, Beschluss vom 2. Juli 2024 – 5 StR 535/23, Rn. 7). Dabei treten Mittäter einheitlich in das Versuchsstadium ein, sobald nur einer von ihnen zu der tatbestandlichen Ausführungshandlung ansetzt (vgl. BGH, Beschlüsse vom 6. September 1989 – 3 StR 268/89, BGHSt 36, 249, 250, und vom 17. März 2022 – 4 StR 223/21, Rn. 15).</text>\n</GermanCourtDecision>\n<GermanCourtDecision>\n  <id_for_citations>jAbAMlOj</id_for_citations>\n  <title>OLG Frankfurt a. M. 1 Ss 80/16</title>\n  <text>Verkündet am: 2016-10-28\nRechtsgebiete: StPR, StR\n9 a) Die Strafrahmenbestimmung bei der Bemessung der Einzelstrafe für die Tat zu Ziff. 1 ist rechtsfehlerhaft erfolgt. Das Amtsgericht hat insoweit einen falschen und für den Angeklagten ungünstigeren Strafrahmen angewandt.\n10 Das Amtsgericht ging für die Tat zu Ziff. 1 von dem Strafrahmen eines Diebstahls im besonders schweren Fall gemäß § 243 Abs. 1 S. 2 Nr. 2 StGB (Freiheitsstrafe von 3 Monaten bis zu 10 Jahren) aus. Dieser Strafrahmen wurde vom Amtsgericht einmal nach §§ 21, 49 Abs. 1 StGB gemildert, da nicht ausgeschlossen werden konnte, dass der Angeklagte bei der Tat aufgrund seiner Drogenabhängigkeit nur vermindert steuerungsfähig war, und ein weiteres Mal nach §§ 23 Abs. 2, 49 Abs. 1 StGB gemildert, da die Tat im Versuch steckenblieb.\n11 Die Annahme eines besonders schweren Falles gemäß § 243 Abs. 1 S. 2 Nr. 2 StGB hält indes aus mehreren Gründen einer rechtlichen Nachprüfung nicht stand.\n12 aa) Hinsichtlich der Tat zu Ziff. 1 kann nicht ausgeschlossen werden, dass sich die Tat gemäß § 243 Abs. 2 StGB auf eine geringwertige Sache bezogen hat, so dass ein besonders schwerer Fall des Diebstahls bereits aus diesem Grund ausscheidet.\n13 Dabei hält der Senat an seiner st. Rspr. fest, nach der die Grenze der Geringwertigkeit i. S. d. §§ 243 Abs. 2; 248a StGB bei 50,- Euro anzusetzen ist (Senat, Beschl. v. 09.05.2008 - 1 Ss 67/08, juris [Rn. 6, 15] = NStZ-RR 2008, 311 m. zust. Anm. und ausf. Begr. bei Jahn, JuS 2008, 1024 m. w. N.; a.A. - freilich unter außendivergenz-hindernder Betonung, dass die Geringwertigkeitsgrenze im Sinne des § 248a StGB nicht starr zu ziehen sei - KG, Beschl. v. 08.01.2015 - [4] 121 Ss 211/14 [276/14], juris [Rn. 18 ff.] = StV 2016, 652, 654 f. = OLGSt StGB § 252 Nr. 3).\n14 Wie die Generalstaatsanwaltschaft zutreffend erkennt, fehlt im amtsgerichtlichen Urteil jegliche Wertangabe bezüglich des Mountain-Bikes. Es finden sich auch keine Feststellungen dazu, um welches Fabrikat es sich handelt oder welches Alter das Mountain-Bike aufweist. Ein Erfahrungssatz, dass fahrtüchtige Mountain-Bikes einen Wert von mehr als 50,- Euro haben, existiert nicht. Auch aus dem Umstand, dass das Amtsgericht bezogen auf die Tat zu Ziff. 2 Ausführungen zur Geringwertigkeit (§ 248a StGB) gemacht hat, kann nichts für die Geringwertigkeitsfrage bei der Tat zu Ziff. 1 abgelesen werden.\n15 bb) Darüber hinaus hat das Amtsgericht die für die Annahme eines besonders schweren Falles gemäß § 243 Abs. 1 StGB gebotene Gesamtwürdigung nicht vorgenommen.</text>\n</GermanCourtDecision>\n<GermanCourtDecision>\n  <id_for_citations>cCzepf8K</id_for_citations>\n  <title>BGH 2 StR 385/10</title>\n  <text>Verkündet am: 2010-08-05\nRechtsgebiete: StPR, StR\nAmtlicher Leitsatz: 1. Der Täter stiehlt auch dann eine durch ein verschlossenes Behältnis besonders gesicherte Sache, wenn er als Unberechtigter den ordnungsgemäß dafür vorgesehenen Schlüssel verwendet. (amtlicher Leitsatz)</text>\n</GermanCourtDecision>\n<GermanCourtDecision>\n  <id_for_citations>ElFWzM3S</id_for_citations>\n  <title>BVerwG 2 WD 3/15</title>\n  <text>Verkündet am: 2015-12-10\nRechtsgebiete: AllgVerwR\n56 Zwar spricht der Umstand, dass der Soldat strafrechtlich nicht Täter (§ 25 StGB) des Eigentumsdelikts zulasten eines Privaten, sondern als Gehilfe (§ 27 StGB) nur Teilnehmer an einer fremden Tat ist, grundsätzlich für ein geringeres Gewicht der Pflichtverletzung und damit für die Annahme eines minderschweren und noch mit einer laufbahnhemmenden Maßnahme angemessen zu sanktionierenden Falles. Allerdings steht hier als Haupttat nicht ein einfacher Diebstahl (§ 242 StGB), sondern ein besonders schwerer Fall des Diebstahls (§ 243 StGB) in Rede, dessen höhere Sanktionsdrohung Ausdruck des im Vergleich zum einfachen Diebstahl höheren Gewichts des Unrechts ist. Die Teilnahme des Soldaten an diesem Delikt zeichnet sich wie oben ausgeführt durch eine besondere Intensität der die Beihilfe begründenden Handlungen und ein eigenes, dem der Täterin gleiches Interesse am Taterfolg aus. Der Umfang seiner Teilnahme erreicht damit zwar noch nicht das Gewicht eines täterschaftlichen Handelns, kommt diesem aber gleichwohl nahe, so dass bereits unter Berücksichtigung dieser Aspekte eine Dienstgradherabsetzung geboten wäre. Rechnung zu tragen ist aber auch dem besonders hohen Schaden, der der Sparkasse entstanden ist. Einen erschwerenden Umstand von hohem, die Verhängung der Höchstmaßnahme rechtfertigenden Gewicht nimmt der Senat bereits bei einer Schadenshöhe im fünfstelligen Bereich an (vgl. BVerwG, Urteil vom 18. April 2013 - 2 WD 16.12 - Buchholz 450.2 § 38 WDO 2002 Nr. 43 Rn. 75). Hier liegt der Schaden im sechsstelligen Bereich. Hinzu tritt außerdem auch noch das Gewicht der im Missbrauch dienstlicher Anlagen als Versteck der Beute liegende Verstoß gegen die innerdienstliche Wohlverhaltenspflicht. Dass der Soldat hiermit über einen langen Zeitraum den besonderen Schutz dienstlicher Anlagen der Bundeswehr zur Sicherung der Beute aus einer Straftat ausnutzte, fällt gravierend zu seinen Lasten ins Gewicht. Bereits unter Berücksichtigung dieser Aspekte haben die erschwerenden Umstände ein so erhebliches Gewicht, dass die Verhängung der Höchstmaßnahme tat- und schuldangemessen ist. Mildernde Umstände müssten ein besonderes Gewicht erreichen, um hiervon absehen zu können (vgl. BVerwG, Urteil vom 15. März 2013 - 2 WD 15.11 - juris Rn. 43 m. w. N.)</text>\n</GermanCourtDecision>\n<GermanCourtDecision>\n  <id_for_citations>gEiUUVWy</id_for_citations>\n  <title>LG Hamburg 712 Ns 95/18</title>\n  <text>Verkündet am: 2019-06-25\ne) Verlassen des Büros durch den Angeklagten\n17 Am Freitag, den 20.6.2014 verließ der Angeklagte das von diesem zusammen seinen ebenfalls im Bereich der Vermögensabschöpfung tätigen Kollegen, den Zeugen d. P. und W., genutzte Büro als letzter um 15:00 Uhr. Dabei ließ er die Bürotür entgegen einer anderslautenden Dienstanweisung seines unmittelbaren Vorgesetzten d. P. offen stehen, um dem Zeugen Y. die Ausführung der Tat zu erleichtern.\n2. Durchführung des Einbruchs\na) Abholung der Tatwerkzeuge durch den Zeugen Y.\n18 Am Samstag, den 21.6.2014 fuhr der Zeuge Y. gegen Abend mit seinem PKW Opel Astra zunächst zum Kleingarten der Ehefrau des Angeklagten im G. M. in H.- B.. Von dort entnahm er aus einem unverschlossenen Werkzeugschuppen die Tatwerkzeuge, die der Angeklagte vereinbarungsgemäß für ihn zur Abholung bereitgestellt hatte. Zusätzlich führte er ein paar sogenannter OP-Handschuhe aus Plastik mit sich, die er bei der Ausführung der Tat unter den vom Angeklagten beschafften Arbeitshandschuhen tragen wollte um sicher zu gehen, dass er am Tatort keine Spuren hinterlassen würde.\nb) Eindringen des Zeugen Y. in das Zollfahndungsamt\n19 Als gegen 20:45 Uhr der letzte Mitarbeiter des Zollfahndungsamtes das Gebäude verlassen und die Alarmanlage scharf geschaltet hatte, traf der Zeuge Y. mit seinem Fahrzeug vor Ort ein. Nachdem er sich eine Weile vor dem Gebäude aufgehalten hatte um sicherzustellen, dass niemand mehr vor Ort war, parkte er in einer Durchfahrt zum Nachbargrundstück des Zollfahndungsamtes und stellte gegen 21:00 Uhr die von ihm mitgeführte Leiter im 1. Stock an der vom B. aus nicht einsehbaren linken Seite des Gebäudes zunächst an einem Fenster in der Mitte des Gebäudes an. Nachdem der Versuch, die dortige Scheibe mit dem Hammer einzuschlagen misslang, verschob er die Leiter weiter nach rechts und unternahm einen erneuten Versuch. Diesmal gelang es ihm, die Scheibe zu zerstören. Mit dem Kuhfuß erweiterte er das Loch und stieg in das Gebäude ein.\nc) Entwenden des Tresors durch den Zeugen Y.</text>\n</GermanCourtDecision>\n<GermanCourtDecision>\n  <id_for_citations>jS0Z3sTo</id_for_citations>\n  <title>LG Landshut 1 KLs 301 Js 13915/17</title>\n  <text>Verkündet am: 2018-03-08\n266 Zu Gunsten des Angeklagten wurde auch in jedem Fall berücksichtigt, dies ist nochmals hervorzuheben, dass die tatsächlich eingetretene Schadenshöhe für den Angeklagten … überraschend gewesen sein dürfte und der Angeklagte mit der Anordnung von hohen Beträgen als Wertersatz für das Taterlangte im Sinne der §§ 73, 73 c StGB belastet ist.\n3. Tat- und schuldangemessene Strafen:\n267 Innerhalb der oben angesprochenen gefundenen Strafrahmen wurden unter Berücksichtigung der vorgenannten für und gegen den Angeklagten sprechenden Umstände, wobei insbesondere der Wert des Diebesguts eine erhebliche Rolle spielte, nicht zuletzt jedoch auch die Beträge berücksichtigt wurden, welche der Angeklagte für die Diebstähle erhielt, folgende Einzelstrafen als tat- und schuldangemessen erachtet:\n268 Im Fall der Gliederungspunkte\nC.I.: jeweils 3 Jahre\nC.II. (Tat vom 24.02.2017): 3 Jahre\nC.II. (Tat vom 15.03.2017): 3 Jahre\nC.III. (Tat vom 05.05.2017): 3 Jahre\nC.III. (Tat vom 10.05.2017): 2 Jahre 9 Monate\n269 Aus diesen verhängten Einzelstrafen war unter Erhöhung der Einsatzstrafe von 3 Jahren gemäß den §§ 53 Abs. 1, 54 StGB eine Gesamtstrafe zu bilden. Dabei waren erneut alle für und gegen den Angeklagten sprechenden Umstände, maßgeblich die vorab bei der Bemessung der Einzelstrafen angeführten, nochmals gegeneinander abzuwägen.\n270 Weiterhin musste zu Gunsten des Angeklagten auch gewertet werden, dass die Taten in einem engen örtlichen und relativ engen zeitlichen Zusammenhang stehen, sie weiterhin von derselben Motivationslage und durch dieselbe Vorgehensweise geprägt sind. Auf der anderen Seite muss auch berücksichtigt werden, dass es sich um mehrere Straftaten handelte, welche sich auch über einen nicht ganz unerheblichen Zeitraum, nämlich über einige Monate hinweg, hinzogen. Im Ergebnis erschien der Kammer eine\nGesamtfreiheitsstrafe von 6 Jahren\nals tat- und schuldangemessen.\nII. Die Strafzumessung bei dem Angeklagten …\n1. Strafrahmen:\n271 Die Kammer entnahm den Strafrahmen jeweils der Vorschrift des § 243 Abs. 1 S. 2 Nr. 3 StGB, welche eine Freiheitsstrafe von 3 Monaten bis zu 10 Jahren vorsieht.\n272 Ein Absehen von dem Regelfall des besonders schweren Falls kam in keinem Fall in Betracht.\n273 In allen Fällen (Tatzeit 24.02.2017 und 15.03.2017) lag ein gewerbsmäßiger Diebstahl nach § 243 Abs. 1 Satz 2 Nr. 3 StGB vor. Auf obige Ausführungen im Rahmen der Beweiswürdigung hinsichtlich der gewerbsmäßigen Begehungsweise wird verwiesen.\n274 Wiederum unterzog die Kammer die Fälle nochmals einer besonderen Überprüfung, da die indizielle Bedeutung durch andere Strafzumessungsfaktoren kompensiert werden kann (Thomas Fischer, StGB, § 46, RdNr. 91).\n275 Hierbei berücksichtigte die Kammer letztendlich wiederum alle wesentlichen Umstände, welche für die Wertung von Tat und Täter in Betracht kamen, gleichgültig, ob sie der Tat innewohnten, sie begleiteten, ihr vorausgingen oder folgten. Letztendlich überzeugte sich die Kammer jedoch davon, dass die indizielle Bedeutung des Regelbeispiels in keinem Fall aufgehoben und der in § 243 Abs. 1 StGB angegebene Strafrahmen für besonders schwere Fälle des Diebstahls gerechtfertigt ist.\n276 Bei dieser Gesamtbetrachtung fielen zugunsten des Angeklagten … folgende wesentliche Gesichtspunkte ins Gewicht:\n277 Der Angeklagte … ist nicht vorbestraft. Die Untersuchungshaft belastet ihn erheblich, da er die deutsche Sprache kaum versteht und somit die Kommunikation in der Haft erheblich eingeschränkt ist. Ferner lebt seine Familie in … und kann daher den Angeklagten nur mit erheblichem Aufwand besuchen.</text>\n</GermanCourtDecision>\n<GermanCourtDecision>\n  <id_for_citations>jZ0hcreP</id_for_citations>\n  <title>LG Hamburg 712 Ns 95/18</title>\n  <text>Verkündet am: 2019-06-25\n20 Im Zollfahndungsamt begab er sich zielgerichtet vom 1. Stock über das Treppenhaus in den durch das Sachgebiet des Angeklagten genutzten 2. Stock, wo er zunächst in dem frei zugänglichen linken Teil des Flures eine von ihm verschlossen vorgefundene Bürotür aufbrach und die Einrichtung des Zimmers in Unordnung brachte, um den Eindruck zu erwecken, dass der Täter keine Kenntnis von den Gegebenheiten im Gebäude gehabt habe. Tatsächlich begab der Zeuge Y. sich sodann zielgerichtet in den durch eine verschlossene Tür gesicherten rechten Teil des Flures, in dem sich - wie er von dem Angeklagten wusste - dessen Büro befand. Hierzu schlug er den Glaseinsatz der Tür ein und brach das Schloss auf. Auf dem Flur öffnete er verschiedene Türen, bis er im Zimmer des Angeklagten den offen auf einem Sideboard platzierten Tresor entdeckte. Dabei handelte es sich einen ca. 20-30 Kilo schweren Möbeltresor. Darin befand sich - verpackt in Plastikbeuteln - im Rahmen der Vermögensabschöpfung sichergestelltes Bargeld in Höhe von insgesamt 130.477,93 € und 30 GBP, darunter auch ein bereits seit dem 30.10.2013 im Tresor verwahrter Einzelbetrag in Höhe von 69.500 €, welcher - wie der Angeklagte wusste - kurz vor der Auskehrung an den Berechtigten durch den zuständigen Zeugen d. P. stand. Mit dem von ihm mitgeführten Kuhfuß hebelte der Zeuge Y. den nur mit zwei Schrauben in der Rigipswand befestigten Safe aus der Wand und warf ihn aus dem Fenster auf die neben dem Gebäude befindliche Rasenfläche. Sodann verließ der Zeuge Y. das Büro des Angeklagten und entnahm aus Neugier, sowie um den Verdacht eines zielgerichteten Vorgehens zu zerstreuen, aus einem am Ende des Ganges befindlichen Materialschrank ein Handfunkgerät der Marke Motorola sowie eine Zollkelle und kehrte mit diesen Gegenständen durch das Treppenhaus in den 1. Stock zurück. Hier stellte er fest, dass er die Flurtür, die sich ohne Schlüssel nur von innen öffnen ließ, und auf die ihn der Angeklagte ihn deshalb besonders hingewiesen hatte, doch hinter sich ins Schloss fallen lassen hatte. Mit dem Hammer schlug er daher unter Einsatz massiver Gewalt den Glaseinsatz der Tür ein und kehrte in das Büro, durch dessen Fenster er eingestiegen war, zurück. Beim Ausstieg aus dem Fenster zog er sich - wie er allerdings erst später feststellte - trotz der von ihm getragenen Handschuhe eine Schnittverletzung an der Hand zu und hinterließ unter anderem am Fensterrahmen und an der Fensterbank eine Blutspur. Unten angekommen, lud er den Tresor in sein Fahrzeug und entfernte sich gegen 21:15 Uhr unter Zurücklassung der Tatwerkzeuge vom Tatort.</text>\n</GermanCourtDecision>\n<GermanCourtDecision>\n  <id_for_citations>QZ2aQ7YH</id_for_citations>\n  <title>OLG Dresden 2 OLG 22 Ss 14/15</title>\n  <text>Verkündet am: 2015-03-12\nRechtsgebiete: StPR, StR\nAmtliche Leitsätze: 1. Zum Ladendiebstahl bei Manipulation des Sicherheitsetiketts am Diebesgut (hier: Entfernung mit Hilfe eines Magneten) (amtlicher Leitsatz) 2. Zu den sachlich-rechtlichen Anforderungen an die Feststellungen zum Tathergang in den Urteilsgründen. Die Darlegung, der Täter habe das Diebesgut "entwendet", reicht ohne den Entwendungsvorgang näher zu beschreiben nicht aus, eine Wegnahme des Gegenstandes festzustellen und die Verurteilung wegen (vollendeten) Diebstahls zu rechtfertigen. (amtlicher Leitsatz)</text>\n</GermanCourtDecision>\n<GermanCourtDecision>\n  <id_for_citations>hYRZenl2</id_for_citations>\n  <title>OLG Frankfurt a. M. 1 Ss 80/16</title>\n  <text>Verkündet am: 2016-10-28\nRechtsgebiete: StPR, StR\nAmtliche Leitsätze: 1. Die Grenze zur Geringwertigkeit einer Sache i. S. der §§ 243 Abs. 2, 248a StGB liegt bei 50,- € (Bestätigung von OLG Frankfurt [1. Strafsenat], NStZ-RR 2008, 311). (amtlicher Leitsatz) 2. Kann nicht ausgeschlossen werden, dass sich die Tat gemäß § 243 Abs. 2 StGB auf eine geringwertige Sache bezogen hat, so scheidet ein besonders schwerer Fall des Diebstahls i. S. von § 243 Abs. 1 StGB aus. (amtlicher Leitsatz) 3. Ein zugebilligter vertypter Strafmilderungsgrund kann - jedenfalls im Zusammenwirken mit den allgemeinen Strafmilderungsgründen - Anlass geben, trotz Vorliegens eines Regelbeispiels einen besonders schweren Fall zu verneinen. Die Darlegungen des Tatrichters müssen erkennen lassen, dass er sich dieser Möglichkeit bewusst ist. (amtlicher Leitsatz)</text>\n</GermanCourtDecision>\n<GermanCourtDecision>\n  <id_for_citations>7msZB42l</id_for_citations>\n  <title>BVerwG 2 WD 3/15</title>\n  <text>Verkündet am: 2015-12-10\nRechtsgebiete: AllgVerwR\nRedaktionelle Leitsätze: 1. Die konkrete Bemessung der Disziplinarmaßnahme erfolgt in einem zweistufigen Prüfungsschema: Auf der ersten Stufe wird eine Regelmaßnahme aus vergleichbaren Fallgruppen hergeleitet, auf der zweiten Stufe wird geprüft, ob im Einzelfall im Hinblick auf die Bemessungskriterien des § 38 Abs. 1 WDO und die Zwecksetzung des Wehrdisziplinarrechts Umstände vorliegen, die die Möglichkeit einer Milderung oder die Notwendigkeit einer Verschärfung gegenüber der auf der ersten Stufe in Ansatz gebrachten Regelmaßnahme eröffnen. 2. In der Summe sind vorliegend so gewichtige erschwerende Umstände (schwerer Diebstahl, Schaden im sechsstelligen Bereich, Missbrauch dienstlicher Anlagen als Versteck der Beute über einen langen Zeitraum, wirtschaftliches Eigeninteresse) gegeben, dass sie auch unter Berücksichtigung der für den Soldaten sprechenden Aspekte (Reue, überobligatorische Wiedergutmachung des Schadens, bisher tadelfreies dienstliches Verhalten) die Entfernung aus dem Dienstverhältnis fordern, weil das Vertrauen in seine Integrität und Zuverlässigkeit zerstört und der Soldat daher für den Dienstherrn objektiv untragbar geworden ist.</text>\n</GermanCourtDecision>\n<GermanCourtDecision>\n  <id_for_citations>XtJHQMpZ</id_for_citations>\n  <title>BGH 1 StR 79/18</title>\n  <text>Verkündet am: 2018-06-26\nRechtsgebiete: StPR, StR\nRedaktionelle Leitsätze: 1. Sicherheitsetiketten an Waren in Kaufhäusern, die akustischen oder optischen Alarm erst auslösen, wenn der Täter das Kaufhaus verlässt, sind keine Schutzvorrichtungen iSd § 243 Abs. 1 S. 2 Nr. 2 StGB, denn sie sind nicht dazu geeignet und bestimmt, den Gewahrsamsbruch, der bei handlichen und leicht beweglichen Sachen in der Regel mit dem Verbergen des Diebesguts in der Kleidung des Täters oder in einem von diesem mitgeführten Behältnis innerhalb des Kaufhauses vollendet ist (vgl. BGH BeckRS 2014, 20034), zu verhindern, sondern sie dienen der Wiederbeschaffung des bereits an den Täter verlorenen Gewahrsams. (Rn. 11) 2. Eine Sicherungsspinne (Elektrodrähte um die Verpackung, die bei Durchtrennung oder beim Passieren des Kassenbereichs ein Alarmsignal auslösen) ist keine Schutzvorrichtungen iSd § 243 Abs. 1 S. 2 Nr. 2 StGB, wenn sie erst bei Verlassen des Geschäfts den Alarm auslöst, deaktiviert oder defekt ist. Hat sie bereits beim Durchtrennen der Drähte Alarm ausgelöst, ist zu prüfen, ob diese Funktion den Bruch des Gewahrsams tatsächlich erschwert. (Rn. 12)</text>\n</GermanCourtDecision>\n<GermanCourtDecision>\n  <id_for_citations>ultJkRVc</id_for_citations>\n  <title>BGH 5 StR 15/20</title>\n  <text>Verkündet am: 2020-04-28\nRechtsgebiete: StPR, StR\nAmtlicher Leitsatz: Versuchsbeginn beim Einbruchdiebstahl. (Rn. 6 – 8) Redaktioneller Leitsatz: Bei einem Diebstahlsdelikt setzt der Täter unmittelbar zur Versuchsbegehung an, wenn aus seiner Sicht die konkrete Gefahr eines ungehinderten Zugriffs auf das in Aussicht genommene Stehlgut bereits besteht. (Rn. 6 – 8)</text>\n</GermanCourtDecision>',
               'name': 'german_court_decision_search',
               'reasoning_content': None,
               'role': 'tool',
               'tool_call_id': 'call_de611e9ce69046018d5ff2db',
               'tool_calls': None},
              {'content': '\n\nNach § 243 I 2 Nr. 2 StGB liegt ein besonders schwerer Fall des Diebstahls in der Regel vor, wenn der Täter eine Sache stiehlt, die durch ein verschlossenes Behältnis oder eine andere Schutzvorrichtung gegen Wegnahme besonders gesichert ist. Die Anforderungen an die Tätigkeit des Täters bei der Überwindung einer solchen besonderen Sicherung sind durch Rechtsprechung und Literatur präzisiert worden.\n\n## Objektive Voraussetzungen der besonderen Sicherung\n\nDie Schutzvorrichtung muss zunächst tatsächlich funktionsfähig und aktiviert sein. Ein offenes Schloss oder ein geöffneter Tresor stellt keine Schutzvorrichtung gegen Wegnahme dar <citation>DTL5LNOq</citation>. Die Sicherung muss nach ihrer erkennbaren Zweckbestimmung dazu dienen, die Wegnahme der Sache zu erschweren und den Gewahrsam des Berechtigten gegen den Bruch durch einen Unbefugten zu sichern <citation>mK9riatq</citation>.\n\nNach der Rechtsprechung des BGH dient § 243 I 2 Nr. 2 StGB der besonderen Sicherung des Diebstahlsobjekts, während § 243 I 2 Nr. 1 StGB besondere Arten der Tatausführung bei einer allgemeinen Sicherung hervorhebt <citation>tou5XNY9</citation><citation>4BwovW5b</citation>.\n\n## Keine besonderen Anforderungen an die Art der Überwindung\n\nFür die Erfüllung des Regelbeispiels kommen auf die konkrete Art und Weise, wie der Täter die Schutzvorrichtung überwindet, keine besonderen Anforderungen an. **Es ist nicht erforderlich, dass der Täter die Sicherung mit Gewalt überwindet oder dass eine bestimmte Kraftentfaltung erfolgt.** Die Rechtsprechung stellt insoweit klar, dass "auf eine besondere Gestaltung der Tathandlung über das Überwinden der Sicherung hinaus bei § 243 Abs. 1 Satz 2 Nr. 2 StGB nicht ankommt" <citation>tou5XNY9</citation>.\n\n### Irrelevanz der Methode der Überwindung\n\nDer BGH hat ausdrücklich entschieden, dass **es nicht darauf ankommt, wie der Täter die Sicherung beseitigt** <citation>tou5XNY9</citation>. Dies wird durch das OLG Hamm bestätigt, wonach "auf eine Kraftentfaltung nicht ankommt, ob und wie der Täter die Schutzvorrichtung beseitigt, ist irrelevant" <citation>NgZaiqGX</citation>.\n\n### Auch rechtmäßig erlangte Mittel können ausreichen\n\nBesonders bedeutsam ist die Erkenntnis, dass selbst die Verwendung des rechtmäßig vorhandenen, aber unbefugt benutzten Schlüssels zur Überwindung der Sicherung ausreichen kann. Der BGH stellt klar:\n\n> "Daher scheidet die Anwendung des Regelbeispiels für einen besonders schweren Fall des Diebstahls wegen der Wegnahme einer Sache aus einem verschlossenen Behältnis auch dann nicht aus, wenn der Verschluss mit dem dafür vorgesehenen Schlüssel geöffnet wird." <citation>tou5XNY9</citation>\n\nVoraussetzung ist hierbei, dass der Täter zur Verwendung des Schlüssels nicht befugt ist. Jedenfalls wenn ein Unberechtigter den Schlüssel an sich nimmt und damit das Behältnis öffnet, überwindet er die Diebstahlssicherung <citation>tou5XNY9</citation>.\n\n## Grenzfälle und Ausnahmen\n\n### Keine wesentliche Erschwerung\n\nEine besondere Sicherung im Sinne des Gesetzes fehlt, wenn die Vorrichtung die Wegnahme nicht wesentlich erschwert. Dies ist z.B. der Fall, wenn der Schlüssel im Schloss steckt, neben dem Behältnis liegt oder leicht erreichbar ist <citation>NgZaiqGX</citation>.\n\n### Keine Schutzvorrichtungen bei reiner Wiedererlangungsfunktion\n\nKeine Schutzvorrichtungen im Sinne des § 243 I 2 Nr. 2 StGB sind Vorrichtungen, die lediglich der nachträglichen Entdeckung des Täters bzw. der Wiedererlangung der gestohlenen Sache dienen sollen. Dazu gehören insbesondere elektronische Sicherungsetiketten an Waren, die erst beim Verlassen des Geschäftsraums einen Alarm auslösen <citation>mK9riatq</citation><citation>XtJHQMpZ</citation>. Solche Etiketten dienen nicht dem Schutz gegen einen Gewahrsamsbruch, sondern nur der Wiedererlangung des bereits an den Täter verlorengegangenen Gewahrsams <citation>mK9riatq</citation>.\n\n### Manipulation statt Überwindung\n\nBei Spielautomaten liegt das Regelbeispiel nur vor, wenn die Sicherungsvorrichtung tatsächlich überwunden wird (z.B. durch Aufbrechen oder regelwidrige Manipulation von außen), nicht aber wenn die Sicherung ohne äußere Manipulation durch Überlistung (z.B. durch Einwurf von Falschgeld) umgangen wird <citation>NgZaiqGX</citation>.\n\n## Besonderheiten beim Versuch\n\nFür den Versuchsbeginn reicht nach der Rechtsprechung der erste Angriff auf einen Schutzmechanismus regelmäßig aus, wenn sich der Täter bei dessen Überwindung nach dem Tatplan ohne tatbestandsfremde Zwischenschritte, zeitliche Zäsur oder weitere eigenständige Willensbildung einen ungehinderten Zugriff auf die erwartete Beute vorstellt <citation>KggPuLF4</citation><citation>ultJkRVc</citation>. Nicht erforderlich ist, dass der angegriffene Schutzmechanismus auch erfolgreich überwunden wird <citation>KggPuLF4</citation>.\n\n## Zusammenfassung\n\nDie Anforderungen an die Tätigkeit des Täters bei der Überwindung einer besonderen Sicherung nach § 243 I 2 Nr. 2 StGB sind vergleichsweiße gering. Entscheidend ist allein, dass der Täter eine tatsächlich funktionsfähige und wirksame Sicherung, die die Wegnahme wesentlich erschwert, irgendeiner Art beseitigt oder umgeht. Die konkrete Methode der Überwindung – ob mit Gewalt, mit dem richtigen Schlüssel, durch Manipulation oder auf sonstige Weise – ist für die Erfüllung des Regelbeispiels grundsätzlich ohne Bedeutung.',
               'name': None,
               'reasoning_content': 'Jetzt habe ich eine gute Sammlung von Kommentaren und Gerichtsentscheidungen, die die Anforderungen an die Tätigkeit des Täters bei der Überwindung einer besonderen Sicherung nach § 243 I 2 Nr. 2 StGB konkretisieren.\n\nDie wichtigsten Erkenntnisse aus den Quellen:\n\n1. Aus dem BGH-Urteil 2 StR 385/10:\n- Es kommt nicht darauf an, wie der Täter die Sicherung überwindet\n- Selbst die Benutzung des richtigen, aber unbefugt erlangten Schlüssels reicht aus\n- § 243 Abs. 1 Satz 2 Nr. 2 StGB betont die besondere Sicherung des Diebstahlsobjekts, während § 243 Abs. 1 Satz 2 Nr. 1 StGB besondere Arten der Tatausführung hervorhebt\n- Auf eine besondere Gestaltung der Tathandlung über das Überwinden der Sicherung hinaus kommt es bei § 243 Abs. 1 Satz 2 Nr. 2 StGB nicht an\n\n2. Aus den Kommentaren:\n- Die Schutzvorrichtung muss die Wegnahme "wesentlich erschweren"\n- Auf eine Kraftentfaltung kommt es nicht an, ob und wie der Täter die Schutzvorrichtung beseitigt, ist irrelevant (OLG Hamm NStZ-RR 2009, 204)\n- Eine besondere Sicherung fehlt, wenn die Vorrichtung die Wegnahme nicht wesentlich erschwert, zB wenn der Schlüssel steckt, neben dem Behältnis liegt oder leicht erreichbar ist\n- Bei Spielautomaten liegt Nr. 2 nur vor, wenn die Sicherungsvorrichtung überwunden wird (zB durch Aufbrechen oder regelwidriger Manipulation von außen), nicht aber ohne äußere Manipulation durch Überlistung umgangen wird\n\n3. Aus anderen Entscheidungen:\n- Die Schutzvorrichtung muss tatsächlich funktionsfähig und aktiviert sein (BGH 1 StR 79/18)\n- Elektronische Sicherungsetiketten sind keine Schutzvorrichtungen i.S.d. § 243 Abs. 1 S. 2 Nr. 2 StGB, weil sie nur der Wiedererlangung des bereits verlorenen Gewahrsams dienen (OLG Dresden 2 OLG 22 Ss 14/15)\n- Für den Versuchsbeginn reicht der erste Angriff auf den Schutzmechanismus regelmäßig aus (BGH 5 StR 15/20)\n\nIch kann diese Erkenntnisse nun strukturiert darstellen und dabei die zitierten Quellen verwenden.',
               'role': 'assistant',
               'tool_call_id': None,
               'tool_calls': []}]}